# ANÁLISIS EXPLORATORIO DE DATOS INCIDENCIA DELICTIVA EN CONTRA DE MUJERES 

In [884]:
# Instalando las paqueterías necesarias.
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install holoviews
%pip install plotly==5.18.0
%pip install unidecode
%pip install scikit-learn
%pip install nbformat


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


...


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


> Las siguientes librerías nos ayudaran para el EDA 

In [885]:
# Importación de librerías para usarlas en el proyecto de descarga y generación del txt con la descripción de esta

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import holoviews as hv
import plotly.express as px
import os
from datetime import datetime
from unidecode import unidecode
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import lognorm
from scipy.optimize import curve_fit
import plotly.subplots as sp
import plotly.graph_objects as go

> La inicialización de ciertas variables ayudaran a preparar la información inicial de cada una de ellas, además de que puedan ser usadas en cualquier parte del código o proceso, sin en el que perjudique el ámbito o espacio donde estas estén siendo seteadas.

In [886]:
# Inicializando variables que serán usadas más adelante

incidencia_clima_link = 'https://open-meteo.com/en/docs/historical-weather-api'
incidencia_title = ''
incidencia_description = ''

folder_path_data = '../rawdata'
folder_path_tidy_data = '../tidydata'
folder_path_log = '../logs/descarga/'
folder_path_error_log = '../logs/errores/'
csv_filepath = ''

# Creando variable que contendra el listado de cada source descargado.
sources_list = []

# Creando variable que contendra el listado de cada nombre de archivo descargado y que esta siendo enviado a nuestro repositorio
names_files_list = []

MergeRelRegFilter = []

texto_list = ''

mes = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
         'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

# Creando un orden personalizado para los meses
orden_meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
               'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

DiccionarioMeses = {
    'Enero':1,
    'Febrero':2,
    'Marzo':3,
    'Abril':4,
    'Mayo':5,
    'Junio':6,
    'Julio':7,
    'Agosto':8,
    'Septiembre':9,
    'Octubre':10,
    'Noviembre':11,
    'Diciembre':12
}

DiccionarioAnios = {
    2013:1,
    2014:2,
    2015:3,
    2016:4,
    2017:5,
    2018:6,
    2019:7,
    2020:8,
    2021:9,
    2022:10,
    2023:11
}

#### Sección donde se encuentran las funciones generales del proceso

In [887]:
def log_error_write(description, line):
    # Verificando si la carpeta existe, si no, crearla
    if not os.path.exists(folder_path_error_log):
        os.makedirs(folder_path_error_log)

    date_now = datetime.now()

    # Variable que contiene el texto que tendrá el archivo txt que se generará y descargará.
    line_error = f"""{date_now.strftime('%Y-%m-%d %H:%M:%S')} {line} - {description}"""
    # Se crea el archivo a partir del texto de arriba, el cual se guardará con el nombre descriptivo más la fecha día*mes*año, para identificar la descarga por día.
    with open(f'{folder_path_error_log}/ddf_err{date_now.strftime("%d%m%y")}.txt', 'a', encoding="utf-8") as error_file:
        error_file.write(f"\n{line_error}")

def log_write(description, line):
    # Verificando si la carpeta existe, si no, crearla
    if not os.path.exists(folder_path_log):
        os.makedirs(folder_path_log)

    date_now = datetime.now()

    # Variable que contiene el texto que tendrá el archivo txt que se generará y descargará.
    line_write = f"""{date_now.strftime('%Y-%m-%d %H:%M:%S')} {line} - {description}"""
    # Se crea el archivo a partir del texto de arriba, el cual se guardará con el nombre descriptivo más la fecha día*mes*año, para identificar la descarga por día.
    with open(f'{folder_path_log}/muncoord{date_now.strftime("%d%m%y")}.txt', 'a', encoding="utf-8") as write_file:
        write_file.write(f"\n{line_write}")


def create_csv_from_DataFrame(df, csv_filename):
    try:

        # Construyendo la ruta completa del archivo
        csv_filepath = os.path.join(folder_path_tidy_data, csv_filename)

        # Guardando el DataFrame como un archivo CSV
        df.to_csv(csv_filepath, index=False, encoding='utf-8')
        print(f"DataFrame (tabla tidy) guardado exitosamente en {csv_filepath}")

    except Exception as e:
        # Imprimiendo un mensaje de error en caso de excepción
        log_error_write(f"Error al guardar el DataFrame (tabla tidy) como CSV: {e}", 'create_csv_from_DataFrame')
        print(f"Error al guardar el DataFrame como CSV: {e}")


def Frecuencia_Relativa_Delito_Region(dfDelictiva, dfClima, municipio, factor_climatologico, extam, anio,  mes, color, grafico):
    try:
        dfDelictivaMunicipio = dfDelictiva[dfDelictiva['municipio'] == municipio ].groupby(['anio','mes'])[['numero_delitos']].agg('sum')
        dfDelictivaMunicipio.reset_index(inplace = True)

        dfClimaMunicipio = dfClima[dfClima['municipio'] == municipio]
        MergeRegion = dfDelictivaMunicipio.merge(dfClimaMunicipio, how = 'inner', on = ['anio','mes'])
        #MergeRegion.head()

        num_total = dfDelictivaMunicipio.groupby('anio')['numero_delitos'].agg('sum')
        #num_total

        MergeRelReg = MergeRegion.merge(num_total, how = 'inner', on = 'anio')
        #MergeRelReg.head()

        MergeRelRegFilter = MergeRelReg

        if(anio > 0):
            MergeRelRegFilter = MergeRelReg[MergeRelReg['anio'] == anio]
    
        if(mes):
            MergeRelRegFilter = MergeRelRegFilter[MergeRelRegFilter['mes'] == mes]

        MergeRelRegFilter['numero_delito_rel'] = MergeRelRegFilter['numero_delitos_x']/MergeRelRegFilter['numero_delitos_y']
        MergeRelRegN = MergeRelRegFilter[MergeRelRegFilter['anio']<2023]

        genera_grafica(grafico, MergeRelRegN, MergeRelRegN[factor_climatologico], MergeRelRegN['numero_delito_rel'], color, f'Temperatura ({factor_climatologico})',
                   'Cantidad de Incidencias', 'Relación entre Temperatura y Incidencias de Delitos', 'Municipio', extam)

        return np.corrcoef(MergeRelRegN[factor_climatologico], MergeRelRegN['numero_delito_rel'])[0][1]
    
    except Exception as e:
        # Imprimiendo un mensaje de error en caso de excepción
        log_error_write(f"Frecuencia_Relativa_Delito_Region: {e}", 'Error al generar el grafico de analisis')
        print(f"Frecuencia_Relativa_Delito_Region: {e}")
        return 0

def Frecuencia_Relativa_Delito_tipo(dfDelictiva, dfClima, delito, factor_climatologico, extam, anio,  mes, color, grafico):
    try:
        dfDelictivaMunicipio = dfDelictiva[dfDelictiva['tipo_delito'] == delito].groupby(['anio','mes'])[['numero_delitos']].agg('sum')
        dfDelictivaMunicipio.reset_index(inplace = True)

        MergeTipoDelito = dfDelictivaMunicipio.merge(dfClima, how = 'inner', on = ['anio','mes'])
        #MergeRegion.head()

        num_total = dfDelictivaMunicipio.groupby('anio')['numero_delitos'].agg('sum')
        #num_total

        MergeRelTipoDel = MergeTipoDelito.merge(num_total, how = 'inner', on = 'anio')
        #MergeRelReg.head()

        MergeRelRegFilter = MergeRelTipoDel

        if(anio > 0):
            MergeRelRegFilter = MergeRelTipoDel[MergeRelTipoDel['anio'] == anio]
    
        if(mes != ""):
            MergeRelRegFilter = MergeRelTipoDel[MergeRelTipoDel['mes'] == mes]
        
        
        MergeRelRegFilter['numero_delito_rel'] = MergeRelRegFilter['numero_delitos_x']/MergeRelRegFilter['numero_delitos_y']
        MergeRelTipoDelN = MergeRelRegFilter[MergeRelRegFilter['anio']<2023]

        genera_grafica(grafico, MergeRelTipoDelN, MergeRelTipoDelN[factor_climatologico], MergeRelTipoDelN['numero_delito_rel'], color, f'Temperatura ({factor_climatologico})',
                   'Cantidad de Incidencias', 'Relación entre Temperatura y Incidencias de Delitos', 'Tipo delito', extam)
        

        return np.corrcoef(MergeRelTipoDelN[factor_climatologico], MergeRelTipoDelN['numero_delito_rel'])[0][1]
    
    except Exception as e:
        # Imprimiendo un mensaje de error en caso de excepción
        log_error_write(f"Frecuencia_Relativa_Delito_tipo: {e}", 'Error al generar el grafico de analisis')
        print(f"Frecuencia_Relativa_Delito_tipo: {e}")
        return 0

def genera_grafica(tipo,df, ejex, ejey, color, titlex, titley, titlegral, legend, dtickc):

    if(tipo == 'box'):
       # Creando boxplot a partir de un dataframe que descarta la incidencia delictica en 0
        #df_grafica = obtener_orden_anio_mes(df)

        if(color != ''):
            figbox = px.box(df, x=ejex, y=ejey, color=color)
        else:
            figbox = px.box(df, x=ejex, y=ejey)

        # Personalizando las etiquetas de los ejes x e y
        figbox.update_xaxes(title_text=titlex)
        figbox.update_yaxes(title_text=titley)

        #fig.update_xaxes(categoryorder="total descending")  # Ordenando las categorías de x por la suma de "numero_delitos"
        figbox.update_layout(xaxis=dict(tickmode="array", tickvals=list(range(len(orden_meses))), ticktext=orden_meses))

        # Agregando un título al gráfico
        #fig.update_layout(title="Distribución de Delitos por Región - Año 2013")

        # Otros ajustes visuales
        figbox.update_traces(boxpoints="all", jitter=0.3, pointpos=-2)  # Mostrando todos los puntos y agregar jitter
        #fig.update_layout(showlegend=False)  # Ocultando la leyenda si no es necesaria

        figbox.show()
    elif(tipo == 'scatter'):
        df_grafica = obtener_orden_anio_mes(df)
         # Creando un gráfico de dispersión
        figscatter = px.scatter(df_grafica, x=ejex, y=ejey)

        # Personalizando las etiquetas de los ejes
        figscatter.update_xaxes(title_text=titlex)
        figscatter.update_yaxes(title_text=titley)
        figscatter.update_layout(title=titlegral, legend_title_text=legend, legend=dict(x=1, y=1), width=900)

        figscatter.show()
    else:
        # Imprimiendo un mensaje de error en caso de excepción
        log_error_write(f"Grafico inexitente", 'El grafico que se quiere generar no existe')
        print(f"Grafico inexitente")

def obtener_orden_anio_mes(df):
    df['código'] = df['anio'].map(lambda x: 12*(DiccionarioAnios[x]-1)) + df['mes'].map(lambda x: DiccionarioMeses[x])
    return df.sort_values('código')


def serie_tiempo_delitos(dfEstatal, delitos):

    dfGrouped = dfEstatal.groupby(['anio','mes','tipo_delito'])['numero_delitos'].agg('sum').reset_index()
    dfGrouped = dfGrouped[dfGrouped['tipo_delito'].isin(delitos)]
    dfGrouped = dfGrouped.set_index(['anio','mes'])

    dfPivot = dfGrouped.pivot(columns = 'tipo_delito', values = 'numero_delitos')
    dfPivot.reset_index(inplace = True)

    dfPivot['periodo'] = dfPivot['anio'].map(lambda x: 12*(DiccionarioAnios[x]-1)) + dfPivot['mes'].map(lambda x: DiccionarioMeses[x])
    dfPivot.sort_values('periodo', inplace = True)

    fig = px.line(dfPivot, x = 'periodo', y = delitos)
    fig.show()

#### Sección para la carga de los datos ha utilizar

In [888]:
try:
    ## Leemos el archivo csv y obtenemos los datos de Incidencias delictivas contra mujeres por tipo de delito
    Incidencia_Delictiva_Mujeres_Tipo_Delito_df = pd.read_csv(f'{folder_path_tidy_data}/Incidencia_Delictiva_Mujeres_Tipo_Delito.csv')

    ## Leemos el archivo csv y obtenemos los datos de incidencias delictivas contra mujeres por municipio
    Incidencia_Delictiva_Mujeres_Municipio_df = pd.read_csv(f'{folder_path_tidy_data}/Incidencia_Delictiva_Mujeres_Municipio.csv')

    ## Leemos el archivo csv y obtenemos los datos climatologicos, esto para poder hacer analisis exploratorio relacionandolo con las cifras de incidencia contra mujeres.
    # Estos datos no se les hara el analisis estadistico, debido a que este estara en la parte del EDA que le corresponde en otra libreta jupiter.
    clima_municipal_df = pd.read_csv(f'{folder_path_tidy_data}/Meteorologia_Climatologia_Municipal.csv')
    clima_estatal_df = pd.read_csv(f'{folder_path_tidy_data}/Meteorologia_Climatologia_Estatal.csv')
    
    print(Incidencia_Delictiva_Mujeres_Tipo_Delito_df.head())
    print(Incidencia_Delictiva_Mujeres_Municipio_df.head())
    print(clima_municipal_df.head())
    print(clima_estatal_df.head())
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error al leer el archivo CSV: {e}", 'Lectura de CifrasIncidenciaDelictivaContraMujeresIndicador')
    print(f"Error al leer el archivo CSV: {e}")

   anio    mes                       tipo_delito  numero_delitos
0  2013  Enero                Violencia Familiar           167.0
1  2013  Enero                       Negligencia            25.0
2  2013  Enero                      Abuso Físico            26.0
3  2013  Enero  Incumpl. obligaciones familiares            31.0
4  2013  Enero               Omisión de Cuidados            15.0
   anio    mes              Municipio  numero_delitos
0  2013  Enero                 Sonora           284.0
1  2013  Enero             Hermosillo           162.0
2  2013  Enero                Nogales            28.0
3  2013  Enero                 Cajeme            25.0
4  2013  Enero  San Luis Rio Colorado            10.0
   anio    mes entidad         region    municipio  temperatura_maxima  \
0  2015  Enero  Sonora     Río Sonora      Aconchi               22.37   
1  2015  Enero  Sonora    Sierra Alta  Agua Prieta               18.09   
2  2015  Enero  Sonora            Sur       Alamos              

In [889]:
# Comprobando valores faltantes en los dataframes ha utilizar
print(Incidencia_Delictiva_Mujeres_Tipo_Delito_df.isna().sum())
print(Incidencia_Delictiva_Mujeres_Municipio_df.isna().sum())
print(clima_municipal_df.isna().sum())
print(clima_estatal_df.isna().sum())

anio              0
mes               0
tipo_delito       0
numero_delitos    0
dtype: int64
anio              0
mes               0
Municipio         0
numero_delitos    0
dtype: int64
anio                      0
mes                       0
entidad                   0
region                  420
municipio                 0
temperatura_maxima        0
temperatura_minima        0
temperatura_promedio      0
salida_sol                0
puesta_sol                0
minutos_luz               0
precipitacion_total       0
lluvia_total              0
tiempo_precipitacion      0
dtype: int64
anio                    0
mes                     0
entidad                 0
temperatura_maxima      0
temperatura_minima      0
temperatura_promedio    0
salida_sol              0
puesta_sol              0
precipitacion_total     0
lluvia_total            0
tiempo_precipitacion    0
minutos_luz             0
dtype: int64


In [890]:
# Comprobando valores nulos en los dataframes ha utilizar
print(Incidencia_Delictiva_Mujeres_Tipo_Delito_df.isnull().sum())
print(Incidencia_Delictiva_Mujeres_Municipio_df.isnull().sum())
print(clima_municipal_df.isnull().sum())
print(clima_estatal_df.isnull().sum())

anio              0
mes               0
tipo_delito       0
numero_delitos    0
dtype: int64
anio              0
mes               0
Municipio         0
numero_delitos    0
dtype: int64
anio                      0
mes                       0
entidad                   0
region                  420
municipio                 0
temperatura_maxima        0
temperatura_minima        0
temperatura_promedio      0
salida_sol                0
puesta_sol                0
minutos_luz               0
precipitacion_total       0
lluvia_total              0
tiempo_precipitacion      0
dtype: int64
anio                    0
mes                     0
entidad                 0
temperatura_maxima      0
temperatura_minima      0
temperatura_promedio    0
salida_sol              0
puesta_sol              0
precipitacion_total     0
lluvia_total            0
tiempo_precipitacion    0
minutos_luz             0
dtype: int64


#### Obtención de datos de las regiones de sonora que esta relacionado con los municipios

In [891]:
try:
    ## Leemos el archivo csv y obtenemos los datos de las distintas regiones en las que esta divido el estado de Sonora
    municipios_regiones = pd.read_csv(f'{folder_path_data}/municipios_regiones.csv')
    municipios_regiones.rename(columns = {'Municipio': 'municipio'}, inplace = True)

    print(municipios_regiones.info())
    print(municipios_regiones.isna().sum())
    print(municipios_regiones['Region'].unique())
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error al leer el archivo CSV: {e}", 'Lectura de las regiones de municipios')
    print(f"Error al leer el archivo CSV: {e}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   municipio  72 non-null     object
 1   Region     72 non-null     object
dtypes: object(2)
memory usage: 1.3+ KB
None
municipio    0
Region       0
dtype: int64
['Noraeste' 'Centro Norte' 'Sur' 'Río Sonora' 'Sierra Alta'
 'Sierra Centro']


#### Primer proceso de limpieza y reorganización de la estructura original para preparación de los datos

In [892]:
try:
  if not municipios_regiones.empty and not Incidencia_Delictiva_Mujeres_Municipio_df.empty:
    # Haciendo un ajustes al dataframe de indicidencia delicitva contra mujeres, los cuales son complementarios al previo proceso de limpieza.
    Incidencia_Delictiva_Mujeres_Municipio_df.rename(columns = {'Municipio': 'municipio'}, inplace = True)
    Incidencia_Delictiva_Mujeres_Municipio_df = Incidencia_Delictiva_Mujeres_Municipio_df[Incidencia_Delictiva_Mujeres_Municipio_df['municipio'].str.lower() != 'sonora'].reset_index(drop=True)

    # Normalizando las columnas en comun (municipio) antes de hacer el merge y agregar al columna region a la tabla tidy
    Incidencia_Delictiva_Mujeres_Municipio_df['municipio_normalizada'] = Incidencia_Delictiva_Mujeres_Municipio_df['municipio'].apply(unidecode)
    municipios_regiones['municipio_normalizada'] = municipios_regiones['municipio'].apply(unidecode)
    Incidencia_Delictiva_Mujeres_Municipio_df['municipio_normalizada'] = Incidencia_Delictiva_Mujeres_Municipio_df['municipio_normalizada'].str.lower()
    municipios_regiones['municipio_normalizada'] = municipios_regiones['municipio_normalizada'].str.lower()
    
    Incidencia_Delictiva_Mujeres_Municipio_df = pd.merge(Incidencia_Delictiva_Mujeres_Municipio_df, municipios_regiones, on='municipio_normalizada', how='left')

    Incidencia_Delictiva_Mujeres_Municipio_df.drop(['municipio_normalizada', 'municipio_x'], axis=1, inplace=True)
    Incidencia_Delictiva_Mujeres_Municipio_df.rename(columns = {'municipio_y': 'municipio'}, inplace = True)
    Incidencia_Delictiva_Mujeres_Municipio_df.rename(columns = {'Region': 'region'}, inplace = True)

    Incidencia_Delictiva_Mujeres_Municipio_df.insert(2, 'region', Incidencia_Delictiva_Mujeres_Municipio_df.pop('region'))
    Incidencia_Delictiva_Mujeres_Municipio_df.insert(3, 'municipio', Incidencia_Delictiva_Mujeres_Municipio_df.pop('municipio'))
  else:
      log_error_write(f"DataFrame leido vacio", 'Lectura vacios de Incidencia_Delictiva_Municipal_df')
      print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error en la limpieza de dataframe: {e}", 'Limpieza Incidencia_Delictiva_Municipal_df 3')
    print(f"Error en la limpieza de dataframe: {e}")

In [893]:
print(Incidencia_Delictiva_Mujeres_Tipo_Delito_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2960 entries, 0 to 2959
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   anio            2960 non-null   int64  
 1   mes             2960 non-null   object 
 2   tipo_delito     2960 non-null   object 
 3   numero_delitos  2960 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 92.6+ KB
None


> Se obverva en el dataframe anterior que la información que se maneja para incidencia delictiva contra mujeres por tipo de delito cuenta con dos caracteristicas categoricas:
> * Mes (Se usa como llave para hacer joins entre tablas)
> * Tipo de delito
>
> Ademas de dos caracteristicas numericas
> * Año (Se usa como llave para hacer joins entre tablas)
> * Numero de delitos (Nuestra variable para generar metricas y calculos, ademas de la caracteristica valor)
>
> Se observa que se tiene un total de 2960 registros (Teniendo en cuenta que cada registro es la combinación de año, mes y tipo de delito)

In [894]:
print(Incidencia_Delictiva_Mujeres_Municipio_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9216 entries, 0 to 9215
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   anio            9216 non-null   int64  
 1   mes             9216 non-null   object 
 2   region          9216 non-null   object 
 3   municipio       9216 non-null   object 
 4   numero_delitos  9216 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 360.1+ KB
None


> Se obverva en el dataframe anterior que la información que se maneja para incidencia delictiva contra mujeres por municipio del estado de Sonora cuenta con tres caracteristicas categoricas:
> * Mes (Se usa como llave para hacer joins entre tablas)
> * Región 
> * Municipio (Se usa como llave para hacer joins entre tablas)
>
> Ademas de dos caracteristicas numericas
> * Año (Se usa como llave para hacer joins entre tablas)
> * Numero de delitos (Nuestra variable para generar metricas y calculos, ademas de la caracteristica valor)
>
> Se observa que se tiene un total de 9216 registros (Teniendo en cuenta que cada registro es la combinación de año, mes y municipio, la region no se toma en cuenta puesto que es una caracteristica relacionada con municipio)

In [895]:
print(clima_municipal_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7560 entries, 0 to 7559
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   anio                  7560 non-null   int64  
 1   mes                   7560 non-null   object 
 2   entidad               7560 non-null   object 
 3   region                7140 non-null   object 
 4   municipio             7560 non-null   object 
 5   temperatura_maxima    7560 non-null   float64
 6   temperatura_minima    7560 non-null   float64
 7   temperatura_promedio  7560 non-null   float64
 8   salida_sol            7560 non-null   object 
 9   puesta_sol            7560 non-null   object 
 10  minutos_luz           7560 non-null   float64
 11  precipitacion_total   7560 non-null   float64
 12  lluvia_total          7560 non-null   float64
 13  tiempo_precipitacion  7560 non-null   float64
dtypes: float64(7), int64(1), object(6)
memory usage: 827.0+ KB
None


> Se obverva en el dataframe anterior que la información que se maneja para incidencia delictiva contra mujeres por municipio del estado de Sonora cuenta con varias caracteristicas categoricas:
> * Mes (Se usa como llave para hacer joins entre tablas)
> * Entidad (Que actualmente siempre sera el Estado de Sonora) 
> * Región 
> * Municipio (Se usa como llave para hacer joins entre tablas)
> * Salida sol 
> * Puesta Sol 
>
> Ademas de dos caracteristicas numericas
> * Año (Se usa como llave para hacer joins entre tablas)
> * Temperatura maxima
> * Temperatura minima
> * Temperatura promedio
> * Temperatura minutos de luz
> * Temperatura Precipitación Total
> * Temperatura Lluvia Total
> * Tiempo precipitación
>
> Se observa que se tiene un total de 525 registros (Teniendo en cuenta que cada registro es la combinación de año, mes y municipio, la region no se toma en cuenta puesto que es una caracteristica relacionada con municipio)

In [896]:
print(clima_estatal_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   anio                  105 non-null    int64  
 1   mes                   105 non-null    object 
 2   entidad               105 non-null    object 
 3   temperatura_maxima    105 non-null    float64
 4   temperatura_minima    105 non-null    float64
 5   temperatura_promedio  105 non-null    float64
 6   salida_sol            105 non-null    object 
 7   puesta_sol            105 non-null    object 
 8   precipitacion_total   105 non-null    float64
 9   lluvia_total          105 non-null    float64
 10  tiempo_precipitacion  105 non-null    float64
 11  minutos_luz           105 non-null    float64
dtypes: float64(7), int64(1), object(4)
memory usage: 10.0+ KB
None


> Se obverva en el dataframe anterior que la información que se maneja para incidencia delictiva contra mujeres por municipio del estado de Sonora cuenta con varias caracteristicas categoricas:
> * Mes (Se usa como llave para hacer joins entre tablas)
> * Entidad (Que actualmente siempre sera el Estado de Sonora) 
> * Salida sol 
> * Puesta Sol 
>
> Ademas de dos caracteristicas numericas
> * Año (Se usa como llave para hacer joins entre tablas)
> * Temperatura maxima
> * Temperatura minima
> * Temperatura promedio
> * Temperatura minutos de luz
> * Temperatura Precipitación Total
> * Temperatura Lluvia Total
> * Tiempo precipitación
>
> Se observa que se tiene un total de 105 registros (Teniendo en cuenta que cada registro es la combinación de año y mes, la region no se toma en cuenta puesto que es una caracteristica relacionada con municipio)

#### Creación de Dataframe de apoyo con los distintos meses del año por columna (Pivot) - Por Tipo de delito

> Se realiza un pivote de incidencia delicitva en contra de mujeres por tipo de delito, esto para apoyarnos en la generación de estadistica y EDA con algunos graficos que nos puede ser de ayuda que la información.
> Cada mes del año que esta por columnas tendra la cantidad de delitos cometidos en el mismo.

In [897]:
# Se realiza un pivot a la tabla para pasar los datos de la columna a mes a distintas columnas y dejar como caracteristica a cada mes del año.

Incidencia_Delictiva_Mujeres_Tipo_Delito_pivot_mes_df = Incidencia_Delictiva_Mujeres_Tipo_Delito_df.pivot(index=["anio", "tipo_delito"], columns="mes", values="numero_delitos").reset_index()
Incidencia_Delictiva_Mujeres_Tipo_Delito_pivot_mes_df = Incidencia_Delictiva_Mujeres_Tipo_Delito_pivot_mes_df[['anio', 'tipo_delito'] + orden_meses]
Incidencia_Delictiva_Mujeres_Tipo_Delito_pivot_mes_df.columns.name = None

print(Incidencia_Delictiva_Mujeres_Tipo_Delito_pivot_mes_df.head(10))

   anio                       tipo_delito  Enero  Febrero  Marzo  Abril  Mayo  \
0  2013                          Abandono    2.0      2.0    0.0    0.0   2.0   
1  2013                       Abuso Fetal    2.0      0.0    0.0    0.0   0.0   
2  2013                      Abuso Físico   26.0     19.0   19.0   45.0  30.0   
3  2013                 Abuso Psicológico    0.0      0.0    3.0    1.0   2.0   
4  2013                      Abuso Sexual    4.0      0.0    2.0    5.0   5.0   
5  2013                Abusos Deshonestos    2.0      0.0    1.0    0.0   3.0   
6  2013             Corrupción de Menores    1.0      0.0    0.0    0.0   0.0   
7  2013                      Drogadicción    0.0      0.0    0.0    0.0   0.0   
8  2013                  Homicidio Doloso    3.0      3.0    5.0    0.0   5.0   
9  2013  Incumpl. obligaciones familiares   31.0     26.0   21.0   22.0  28.0   

   Junio  Julio  Agosto  Septiembre  Octubre  Noviembre  Diciembre  
0    2.0    1.0     3.0         2.0    

#### Creación de Dataframe de apoyo con los distintos años involucrados por columna (Pivot) - Por Tipo de delito

> Se realiza un pivote de incidencia delicitva en contra de mujeres por tipo  de delito, esto para apoyarnos en la generación de estadistica y EDA con algunos graficos que nos puede ser de ayuda que la información.
>
> Cada año esta en cada columna, estas tendran la cantidad de delitos cometidos en el mismo.

In [898]:
# Se realiza un pivot a la tabla para pasar los datos de la columna a mes a distintas columnas y dejar como caracteristica a cada año que tiene registro

Incidencia_Delictiva_Mujeres_Tipo_Delito_pivot_anio_df = Incidencia_Delictiva_Mujeres_Tipo_Delito_df.pivot(index=["mes","tipo_delito"], columns="anio", values="numero_delitos").reset_index()
Incidencia_Delictiva_Mujeres_Tipo_Delito_pivot_anio_df.columns.name = None
Incidencia_Delictiva_Mujeres_Tipo_Delito_pivot_anio_df.fillna(0, inplace=True)

print(Incidencia_Delictiva_Mujeres_Tipo_Delito_pivot_anio_df.head(10))

     mes              tipo_delito  2013  2014  2015  2016  2017  2018  2019  \
0  Abril                 Abandono   0.0   0.0   3.0   0.0   4.0   0.0   1.0   
1  Abril              Abuso Fetal   0.0   0.0   0.0   0.0   1.0   1.0   1.0   
2  Abril             Abuso Físico  45.0  13.0  15.0  33.0  25.0   4.0   0.0   
3  Abril        Abuso Psicológico   1.0   1.0   1.0   1.0  15.0  17.0   7.0   
4  Abril             Abuso Sexual   5.0   4.0   3.0   2.0   6.0   3.0   8.0   
5  Abril       Abusos Deshonestos   0.0   0.0   1.0   9.0  19.0  15.0   4.0   
6  Abril  Acoso escolar(Bullying)   0.0   0.0   0.0   0.0   2.0   0.0   0.0   
7  Abril            Acoso laboral   0.0   0.0   0.0   1.0   1.0   0.0   0.0   
8  Abril              Alcoholismo   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
9  Abril    Corrupción de Menores   0.0   1.0   1.0   1.0   2.0   0.0   0.0   

   2020  2021  2022  2023  
0   1.0   0.0   3.0   0.0  
1   2.0   0.0   0.0   0.0  
2   4.0  11.0  12.0  10.0  
3   1.0  64.0  52.

#### Creación de Dataframe de apoyo con los distintos meses del año por columna (Pivot) - Por Municipio

> Se realiza un pivote de incidencia delicitva en contra de mujeres por municipio, esto para apoyarnos en la generación de estadistica y EDA con algunos graficos que nos puede ser de ayuda que la información.
>
> Cada mes del año que esta por columnas tendra la cantidad de delitos cometidos en el mismo.

In [899]:
# Se realiza un pivot a la tabla para pasar los datos de la columna a mes a distintas columnas y dejar como caracteristica a cada mes del año.

Incidencia_Delictiva_Mujeres_Municipio_pivot_mes_df = Incidencia_Delictiva_Mujeres_Municipio_df.pivot(index=["anio", "region", "municipio"], columns="mes", values="numero_delitos").reset_index()
Incidencia_Delictiva_Mujeres_Municipio_pivot_mes_df = Incidencia_Delictiva_Mujeres_Municipio_pivot_mes_df[['anio', 'region', 'municipio'] + orden_meses]
Incidencia_Delictiva_Mujeres_Municipio_pivot_mes_df.columns.name = None

print(Incidencia_Delictiva_Mujeres_Municipio_pivot_mes_df.head(10))

   anio        region                 municipio  Enero  Febrero  Marzo  Abril  \
0  2013  Centro Norte             Benjamín Hill    1.0      1.0    0.0    1.0   
1  2013  Centro Norte                     Carbó    0.0      0.0    0.0    0.0   
2  2013  Centro Norte                   Cucurpe    0.0      0.0    0.0    0.0   
3  2013  Centro Norte                Hermosillo  162.0    167.0  146.0  198.0   
4  2013  Centro Norte                 Magdalena    6.0      0.0    3.0    0.0   
5  2013  Centro Norte                   Nogales   28.0     21.0   25.0   27.0   
6  2013  Centro Norte                   Opodepe    0.0      0.0    0.0    0.0   
7  2013  Centro Norte  San Miguel de Horcasitas    0.0      0.0    0.0    0.0   
8  2013  Centro Norte                 Santa Ana    0.0      0.0    0.0    0.0   
9  2013  Centro Norte                Santa Cruz    0.0      0.0    0.0    0.0   

    Mayo  Junio  Julio  Agosto  Septiembre  Octubre  Noviembre  Diciembre  
0    0.0    0.0    0.0     2.0  

#### Creación de Dataframe de apoyo con los distintos años involucrados por columna (Pivot) - Por Municipio

> Se realiza un pivote de incidencia delicitva en contra de mujeres por tipo  de delito, esto para apoyarnos en la generación de estadistica y EDA con algunos graficos que nos puede ser de ayuda que la información.
>
> Cada año esta en cada una de las columnas teniendo la cantidad de delitos cometidos en el mismo.

In [900]:
Incidencia_Delictiva_Mujeres_Municipio_pivot_anio_df = Incidencia_Delictiva_Mujeres_Municipio_df.pivot(index=["mes","region", "municipio"], columns="anio", values="numero_delitos").reset_index()
Incidencia_Delictiva_Mujeres_Municipio_pivot_anio_df.columns.name = None

print(Incidencia_Delictiva_Mujeres_Municipio_pivot_anio_df.head(10))

     mes        region                 municipio   2013   2014   2015   2016  \
0  Abril  Centro Norte             Benjamín Hill    1.0    0.0    0.0    0.0   
1  Abril  Centro Norte                     Carbó    0.0    0.0    0.0    0.0   
2  Abril  Centro Norte                   Cucurpe    0.0    0.0    0.0    0.0   
3  Abril  Centro Norte                Hermosillo  198.0  146.0  163.0  158.0   
4  Abril  Centro Norte                 Magdalena    0.0    2.0    0.0    2.0   
5  Abril  Centro Norte                   Nogales   27.0   17.0   14.0   13.0   
6  Abril  Centro Norte                   Opodepe    0.0    0.0    0.0    0.0   
7  Abril  Centro Norte  San Miguel de Horcasitas    0.0    0.0    0.0    0.0   
8  Abril  Centro Norte                 Santa Ana    0.0    0.0    0.0    0.0   
9  Abril  Centro Norte                Santa Cruz    0.0    0.0    0.0    0.0   

    2017   2018   2019   2020   2021   2022   2023  
0    1.0    2.0    0.0    0.0    0.0    0.0    0.0  
1    0.0    0

## Estadística

> En esta sección se realiza el analisis estadistico de las distintas tablas o dataframes que se han obtenido previamente.
>
> Contiene datos estadisticos y graficas que ayudan a la etapa EDA.

#### Creando listado de los años involucrados en los datos obtenidos de incidencia delictiva

In [901]:
# Creando una serie de todos los años involucrados en los datos de incidencia delictiva en contra de las mujeres agregandolos como lista
lista_anio = Incidencia_Delictiva_Mujeres_Tipo_Delito_df["anio"].unique()
print(lista_anio)

# Creando la lista de años en formato string, esto para que sirva al crear los titulos de las diferentes graficas cuando se utilice subplots
lista_anio_string = []
for year in lista_anio:
    lista_anio_string.append(str(year))

# Creando listado de tres años con relevancia para objeto de comparación
# Se toma el primer año de estudio, el anterior al actual para poder obtener la data completa y el año actual a pesar de no contar con todos los datos o meses disponibles aun.
# El ultimo año se toma como referencia para ir viendo como va actualmente la incidencia a comparación del año anterior o el primer año.
lista_anios_especificos = [min(lista_anio), max(lista_anio) - 1, max(lista_anio)]

# Creando la lista de años en formato string, esto para que sirva al crear los titulos de las diferentes graficas cuando se utilice subplots
lista_anio_especificos_string = []
for year in lista_anios_especificos:
    lista_anio_especificos_string.append(str(year))

print(lista_anios_especificos)

lista_mes = Incidencia_Delictiva_Mujeres_Tipo_Delito_df['mes'].unique()

print(lista_mes)

[2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023]
[2013, 2022, 2023]
['Enero' 'Febrero' 'Marzo' 'Abril' 'Mayo' 'Junio' 'Julio' 'Agosto'
 'Septiembre' 'Octubre' 'Noviembre' 'Diciembre']


### Histogramas frecuencia en numero de delitos por año.

#### Serie de Histogramas que representan la frecuencia con el que un numero de delito en cada año (Considerando el valor 0 incidencias)

In [902]:
# Creando un subplot con dos columnas por fila
filas = (len(lista_anio) + 1) // 2
columnas = 2

titulo_personalizado = [f"Año {year}" for year in lista_anio_string]

fig = sp.make_subplots(rows=filas, cols=columnas, subplot_titles=titulo_personalizado)

# Rellenando los subplots de cada año para mostrar la frecuencia de numero de incidencia en cada uno
for i, year in enumerate(lista_anio):
    datos = Incidencia_Delictiva_Mujeres_Tipo_Delito_df[Incidencia_Delictiva_Mujeres_Tipo_Delito_df['anio'] == year]['numero_delitos'] # Seleccionando los datos del año específico
    histograma = go.Histogram(x=datos, name=str(year))
    
    # Agregando el histograma al subplot correspondiente
    fila = i // columnas + 1
    columna = i % columnas + 1
    fig.add_trace(histograma, row=fila, col=columna)

# Personalizando el diseño y las etiquetas de los ejes
fig.update_xaxes(title_text="Número de Delitos")
fig.update_yaxes(title_text="Frecuencia")
fig.update_layout(height=250 * filas, title_text="Histogramas para Años 2013-2023")
fig.show()

> Los histogramas anteriores mostrados, presentan la información siguiente:
>   * Se cuenta con una gran cantidad de registros con incidencia 0 en todos los años, esto por que en determinados delitos no se presentaron en determinado año y mes. 
>   * El año 2013, es uno de los mas bajos en cuanto numero de delitos totales.
>   * El año 2019 y 2021, son los que mas cantidad de incidencia delictiva total se ha presentado, aunque la frecuencia de cada valor alto se distribuyen entre cada uno.

#### Histogramas con relevancia para comparación entre distintos años

> Histograma del 2013 por ser el primer año que se tiene registro de los datos de incidencia delictiva en contra de mujeres.
>
> Histograma del año anterior al actual por ser el ultimo año con información completa.
>
> Histograma del ultimo año registrado, es el año actual, pero que puede contener información faltante.

In [903]:
for year in lista_anios_especificos:

    year_delitos = Incidencia_Delictiva_Mujeres_Tipo_Delito_df[Incidencia_Delictiva_Mujeres_Tipo_Delito_df['anio'] == year]
    # Creando el histograma del año
    fig = px.histogram(year_delitos, x="numero_delitos", marginal="rug")
    fig.update_xaxes(title_text="Número de Delitos")
    fig.update_yaxes(title_text="Frecuencia")

    # Agregando título al gráfico
    fig.update_layout(
        title=f"Histograma de Número de Delitos en {str(year)}",
        bargap=0.1
    )

    # Otros ajustes visuales
    fig.update_traces(marker_color="blue")
    fig.update_layout(showlegend=False)

    fig.show()

> Los histogramas anteriores mostrados, presentan la información siguiente:
>   * Distribución del numeros de delitos en el año 2013, donde en el rango entre 20 y 29 delitos fue la frecuencia mas alta que se presento durante dicho año (sin contar 0 incidencia)
>   * Distribución del numeros de delitos en el año 2022, donde en el rango entre 20 y 39 delitos fue la frecuencia mas alta que se presento durante dicho año (sin contar 0 incidencia)
>   * Distribución del numeros de delitos en el año 2023, donde en el rango entre 20 y 39 delitos fue la frecuencia mas alta que se presento durante dicho año (sin contar 0 incidencia), sin embargo es frecuencia es el bajo entre los histogramas que se esta comparando.

#### Serie de Histogramas que representan la frecuencia con el que un numero de delito en cada año (Sin considerar registros con 0 incidencia)

In [904]:
# Se realiza la creación de un histograma sin los valores 0, lo cual quiere decir que no se tomara en cuenta los registros que tengan 0 incidencia delictiva.
Incidencia_Delictiva_Mujeres_Tipo_Delito_Sin_0_df = Incidencia_Delictiva_Mujeres_Tipo_Delito_df[Incidencia_Delictiva_Mujeres_Tipo_Delito_df['numero_delitos']>0]

In [905]:
# Creando un subplot con dos columnas por fila
filas = (len(lista_anio) + 1) // 2
columnas = 2

titulo_personalizado = [f"Año {year}" for year in lista_anio_string]

fig = sp.make_subplots(rows=filas, cols=columnas, subplot_titles=titulo_personalizado)

# Rellenando los subplots de cada año para mostrar la frecuencia de numero de incidencia en cada uno
for i, year in enumerate(lista_anio):
    datos = Incidencia_Delictiva_Mujeres_Tipo_Delito_Sin_0_df[Incidencia_Delictiva_Mujeres_Tipo_Delito_Sin_0_df['anio'] == year]['numero_delitos'] # Selecciona los datos del año específico
    histograma = go.Histogram(x=datos, name=str(year))
    
    # Agregando el histograma al subplot correspondiente
    fila = i // columnas + 1
    columna = i % columnas + 1
    fig.add_trace(histograma, row=fila, col=columna)

# Personalizando el diseño y las etiquetas de los ejes
fig.update_xaxes(title_text="Número de Delitos")
fig.update_yaxes(title_text="Frecuencia")
fig.update_layout(height=250 * filas, title_text="Histogramas para Años 2013-2023")
fig.show()

> Los histogramas anteriores mostrados, presentan la información siguiente:
>   * Se cuenta con una gran cantidad de registros dejando fuera la incidencia 0 en todos los años, a pesar de ello sigue mostrando una frecuencia alta en cantidad de delitos bajos
>   * El año 2013, es uno de los mas bajos en cuanto numero de delitos totales.
>   * El año 2019 y 2021, son los que mas cantidad de incidencia delictiva totales se ha presentado, aunque la frecuencia de cada valor alto se distribuyen entre cada uno de la misma manera.

#### Histogramas con relevancia para comparación entre distintos años

> Histograma del 2013 por ser el primer año que se tiene registro de los datos de incidencia delictiva en contra de mujeres.
>
> Histograma del año anterior al actual por ser el ultimo año con información completa.
>
> Histograma del ultimo año registrado, es el año actual, pero que puede contener información faltante.

In [906]:
for year in lista_anios_especificos:

    year_delitos = Incidencia_Delictiva_Mujeres_Tipo_Delito_Sin_0_df[Incidencia_Delictiva_Mujeres_Tipo_Delito_Sin_0_df['anio'] == year]
    # Creando el histograma para el año
    fig = px.histogram(year_delitos, x="numero_delitos", marginal="rug")

    # Personalizando las etiquetas de los ejes x e y
    fig.update_xaxes(title_text="Número de Delitos")
    fig.update_yaxes(title_text="Frecuencia")

    # Agregando un título al gráfico
    fig.update_layout(
        title=f"Histograma de Número de Delitos en {str(year)}",
        bargap=0.1
    )

    # Otros ajustes visuales
    fig.update_traces(marker_color="blue")
    fig.update_layout(showlegend=False)

    fig.show()

> Los histogramas anteriores mostrados, presentan la información siguiente:
>   * Distribución del numeros de delitos en el año 2013, donde en el rango entre 1 y 19 delitos fue la frecuencia mas alta que se presento durante dicho año
>   * Distribución del numeros de delitos en el año 2022, donde en el rango entre 1 y 19 delitos fue la frecuencia mas alta que se presento durante dicho año, este año fue el que presento mayor frecuencia en ese rango de numero de delitos a comparación con los otros años.
>   * Distribución del numeros de delitos en el año 2023, donde en el rango entre 1 y 19 delitos fue la frecuencia mas alta que se presento durante dicho año

### Sección de estadistica de las caracteristicas numericas de los dataframe de incidencia delictiva

> Se realizan estadistica basica de los diferentes dataframe, para observar a la caracteristica numero_delitos y poder sacar primeras conclusiones
> 
> Información de los numeros de delitos a lo largo de casi 10 años

In [907]:
print(Incidencia_Delictiva_Mujeres_Tipo_Delito_df.describe())

              anio  numero_delitos
count  2960.000000     2960.000000
mean   2018.143243       20.972297
std       2.948428       68.050450
min    2013.000000        0.000000
25%    2016.000000        0.000000
50%    2018.000000        2.000000
75%    2021.000000       10.000000
max    2023.000000      667.000000


> El Datraframe descrito estadisticamente, se recuerda que cuenta con las caracteristicas año, mes, y tipo de delito
>
> * En promedio se comenten alrededor de 20.97 delitos durante el periodo que se esta analizando.
> * La desviación estandar alta nos sugiere que los valores de numero de delitos estan dispersos ampliamente alrededor de nuestra media.
> * Tenemos un valor minimo de 0 indicando que existen observaciones sin incidencias registradas.
> * El 75% de nuestros registros u observaciones en el numero de delitos son iguales o menos de 10.
> * El maximo numero de delitos que se tiene registrado fue de 667 para un determinado año, mes y tipo de delito.

In [908]:
print(Incidencia_Delictiva_Mujeres_Municipio_df.describe())

              anio  numero_delitos
count  9216.000000     9216.000000
mean   2017.843750        6.734592
std       3.083483       29.448998
min    2013.000000        0.000000
25%    2015.000000        0.000000
50%    2018.000000        0.000000
75%    2020.250000        1.000000
max    2023.000000      416.000000


> El Datraframe descrito estadisticamente, se recuerda que cuenta con las caracteristicas año, mes, y municipio
>
> * En promedio se comenten alrededor de 6.73 delitos durante el periodo que se esta analizando. Algo bajo pero puede deberse a la mayoria de registros con cero incidencia delictiva.
> * La desviación estandar es algo alta nos sugiere que los valores de numero de delitos estan dispersos ampliamente alrededor de nuestra media.
> * Tenemos un valor minimo de 0 indicando que existen observaciones sin incidencias registradas.
> * El 75% de nuestros registros u observaciones en el numero de delitos son iguales o menores a 1, muchos registros con numero de delitos realmente bajos.
> * El maximo numero de delitos que se tiene registrado fue de 416 para un determinado año, mes y municipio.

In [909]:
print(clima_estatal_df.describe())

              anio  temperatura_maxima  temperatura_minima  \
count   105.000000          105.000000          105.000000   
mean   2018.885714           29.723619           15.795143   
std       2.539360            5.769238            6.337655   
min    2015.000000           19.920000            5.940000   
25%    2017.000000           24.520000            9.810000   
50%    2019.000000           30.730000           15.580000   
75%    2021.000000           34.210000           22.530000   
max    2023.000000           39.370000           25.990000   

       temperatura_promedio  precipitacion_total  lluvia_total  \
count            105.000000           105.000000    105.000000   
mean              22.661238          2204.018095   2200.029524   
std                6.045750          2761.314481   2761.782170   
min               12.860000             0.200000      0.200000   
25%               17.090000           161.000000    161.000000   
50%               23.450000          1389.600

> Esta estadistica se habla mas a detalle en su documento correspondiente en la ruta /EDA/IC_EDA_Clima.ipynb

In [910]:
print(clima_municipal_df.describe())

              anio  temperatura_maxima  temperatura_minima  \
count  7560.000000         7560.000000         7560.000000   
mean   2018.885714           29.724099           15.794496   
std       2.527407            6.225475            6.798770   
min    2015.000000           13.270000           -0.010000   
25%    2017.000000           24.677500           10.107500   
50%    2019.000000           30.590000           15.630000   
75%    2021.000000           34.680000           21.992500   
max    2023.000000           44.790000           30.280000   

       temperatura_promedio  minutos_luz  precipitacion_total  lluvia_total  \
count           7560.000000  7560.000000          7560.000000   7560.000000   
mean              22.661501   732.037889            30.611362     30.555966   
std                6.429872    77.368975            44.555360     44.545818   
min                6.070000   604.130000             0.000000      0.000000   
25%               17.360000   666.750000      

> Esta estadistica se habla mas a detalle en su documento correspondiente en la ruta /EDA/IC_EDA_Clima.ipynb

### Sección para detectar diferencias entre los distintos dataframe de incidencia delictiva contra mujeres (Tipo de delito vs Municipio)

> Si bien sería algo que a simple lógica no debería pasar el de encontrar diferencias entre las distintas tablas por tipo de delito y municipio, esto hablando en el número de delitos cometidos, es algo que debemos considerar, por lo tanto la siguiente parte trata respecto a encontrar dichas diferencias.

#### Información por municipio

In [911]:
#print(Incidencia_Delictiva_Mujeres_Municipio_df.groupby(['anio', 'mes'])['numero_delitos'].sum())

# se usa pivot_table para obtener un nuevo DataFrame con tan solo las columnas de año, y los distintos meses como las demas columnas de información
consulta_municipio_df = Incidencia_Delictiva_Mujeres_Municipio_df.pivot_table(index='anio', columns='mes', values='numero_delitos', aggfunc='sum', fill_value=0)
# Reiniciando el índice para que 'año' sea una columna en lugar de un índice
consulta_municipio_df = consulta_municipio_df.reset_index()
consulta_municipio_df = consulta_municipio_df[['anio'] + orden_meses]
consulta_municipio_df.set_index('anio', inplace=True)
print(consulta_municipio_df)

mes   Enero  Febrero  Marzo  Abril   Mayo  Junio  Julio  Agosto  Septiembre  \
anio                                                                          
2013  284.0    254.0  256.0  307.0  300.0  290.0  198.0   286.0       240.0   
2014  265.0    252.0  276.0  259.0  290.0  326.0  220.0   242.0       293.0   
2015  275.0    240.0  257.0  246.0  244.0  299.0  179.0   186.0       215.0   
2016  259.0    256.0  272.0  323.0  327.0  487.0  384.0   496.0       460.0   
2017  614.0    553.0  648.0  598.0  654.0  707.0  669.0   720.0       651.0   
2018  501.0    491.0  518.0  529.0  688.0  836.0  697.0   735.0       791.0   
2019  502.0    419.0  515.0  480.0  649.0  790.0  719.0   945.0       809.0   
2020  565.0    575.0  454.0  343.0  362.0  459.0  512.0   610.0       672.0   
2021  631.0    593.0  769.0  749.0  905.0  931.0  816.0   838.0       759.0   
2022  561.0    474.0  591.0  574.0  629.0  680.0  486.0   511.0       627.0   
2023  523.0    431.0  564.0  494.0  561.0  482.0  41

#### Información por tipo de delito

In [912]:
#print(Incidencia_Delictiva_Mujeres_Tipo_Delito_df.groupby(['anio', 'mes'])['numero_delitos'].sum())

# se usa pivot_table para obtener un nuevo DataFrame con tan solo las columnas de año, y los distintos meses como las demas columnas de información
consulta_tipodelito_df = Incidencia_Delictiva_Mujeres_Tipo_Delito_df.pivot_table(index='anio', columns='mes', values='numero_delitos', aggfunc='sum', fill_value=0)
# Reiniciando el índice para que 'año' sea una columna en lugar de un índice
consulta_tipodelito_df = consulta_tipodelito_df.reset_index()
consulta_tipodelito_df = consulta_tipodelito_df[['anio'] + orden_meses]
consulta_tipodelito_df.set_index('anio', inplace=True)
print(consulta_tipodelito_df)

mes   Enero  Febrero  Marzo  Abril   Mayo  Junio  Julio  Agosto  Septiembre  \
anio                                                                          
2013  284.0    257.0  256.0  307.0  300.0  290.0  198.0   286.0       240.0   
2014  265.0    252.0  276.0  259.0  290.0  326.0  220.0   242.0       293.0   
2015  275.0    240.0  257.0  246.0  244.0  299.0  179.0   186.0       215.0   
2016  259.0    256.0  272.0  323.0  327.0  487.0  384.0   496.0       460.0   
2017  615.0    553.0  648.0  599.0  654.0  707.0  669.0   720.0       651.0   
2018  501.0    491.0  518.0  529.0  688.0  837.0  697.0   735.0       791.0   
2019  502.0    419.0  515.0  480.0  649.0  790.0  719.0   945.0       809.0   
2020  565.0    575.0  455.0  343.0  362.0  459.0  512.0   610.0       672.0   
2021  631.0    593.0  769.0  749.0  905.0  931.0  816.0   838.0       759.0   
2022  561.0    474.0  592.0  574.0  629.0  680.0  486.0   511.0       627.0   
2023  523.0    431.0  565.0  494.0  561.0  482.0  41

#### Encontrando las diferencias entre los dataframe entre tipo de delito y Municipio en su columna Numero de delitos (cantidad de incidencias registradas)

In [913]:
#diferencias_totales = consulta_municipio_df.compare(consulta_tipodelito_df, keep_shape=True, keep_equal=True)
# Se utiliza la función compare para realizar la detención de diferencias, previamente se acomodo los dataframe para que tuvieran la misma estructura.
diferencias_totales = consulta_municipio_df.compare(consulta_tipodelito_df)

etiquetas = {
    'self': 'municipioDF',
    'other': 'tipodelitoDF'
}

diferencias_totales = diferencias_totales.rename(columns=etiquetas, level=0)
diferencias_totales = diferencias_totales.rename(columns=etiquetas, level=1)

print(diferencias_totales)

mes        Enero                  Febrero                    Marzo  \
     municipioDF tipodelitoDF municipioDF tipodelitoDF municipioDF   
anio                                                                 
2013         NaN          NaN       254.0        257.0         NaN   
2016         NaN          NaN         NaN          NaN         NaN   
2017       614.0        615.0         NaN          NaN         NaN   
2018         NaN          NaN         NaN          NaN         NaN   
2020         NaN          NaN         NaN          NaN       454.0   
2022         NaN          NaN         NaN          NaN       591.0   
2023         NaN          NaN         NaN          NaN       564.0   

mes                     Abril                    Junio               \
     tipodelitoDF municipioDF tipodelitoDF municipioDF tipodelitoDF   
anio                                                                  
2013          NaN         NaN          NaN         NaN          NaN   
2016          N

> Basandose en los totales del numero de delitos por mes en cada año, se puede apreciar que existen años y meses donde existen diferencias, pero son demasiadas pequeñas diferencia.
>
> Lo interesante aqui es que la diferencia en todas son de una sola unidad, como si en el dataframe de tipo de delitos en algun determinado tipo de delito se estuviera contabilizando uno de mas.

Veces de regiones registradas en el dataframe por municipio

In [914]:
print(Incidencia_Delictiva_Mujeres_Municipio_df["region"].value_counts())

region
Sierra Alta      1920
Sur              1664
Centro Norte     1408
Noraeste         1408
Río Sonora       1408
Sierra Centro    1408
Name: count, dtype: int64


> La region de sierra alta es el que tiene mas registros en el que aparece, luego le sigue la region Sur, el resto comparten la misma cantidad.

Cantidad de delitos por año

In [915]:
print(Incidencia_Delictiva_Mujeres_Municipio_df.groupby(['anio'])['numero_delitos'].sum())

anio
2013    3028.0
2014    3124.0
2015    2792.0
2016    4597.0
2017    7471.0
2018    7423.0
2019    7839.0
2020    6680.0
2021    8793.0
2022    6424.0
2023    3895.0
Name: numero_delitos, dtype: float64


> El año 2021 es el que tiene el mayor numero de delitos cometidos en contra de mujeres.

Cantidad de delitos por mes

In [916]:
print(Incidencia_Delictiva_Mujeres_Municipio_df.groupby(['mes'])['numero_delitos'].sum())

mes
Abril         4902.0
Agosto        5999.0
Diciembre     3484.0
Enero         4980.0
Febrero       4538.0
Julio         5290.0
Junio         6287.0
Marzo         5120.0
Mayo          5609.0
Noviembre     4819.0
Octubre       5521.0
Septiembre    5517.0
Name: numero_delitos, dtype: float64


> El mes con mayor cantidad de delitos es Junio, ¿a caso un mes tan calido esta relacionado con el aumento de delitos?
>
> Diciembre curiosamente tiene la menor cantidad de delitos.

In [917]:
total_delitos = Incidencia_Delictiva_Mujeres_Municipio_df['numero_delitos'].sum()
print(total_delitos)

62066.0


> Ha habido un total de 62,066 delitos contra la mujer en un periodo aprox de 10 años

Cantidad de delitos por region

In [918]:
totales_region = Incidencia_Delictiva_Mujeres_Municipio_df.groupby(['region'])['numero_delitos'].sum().reset_index()
totales_region["porcentaje_respecto_total"] = totales_region['numero_delitos'] / total_delitos
totales_region = totales_region.sort_values(by = 'porcentaje_respecto_total', ascending=False)

print(totales_region)

          region  numero_delitos  porcentaje_respecto_total
0   Centro Norte         34719.0                   0.559388
5            Sur         17616.0                   0.283827
1       Noraeste          6416.0                   0.103374
3    Sierra Alta          1822.0                   0.029356
2     Río Sonora          1175.0                   0.018931
4  Sierra Centro           318.0                   0.005124


> La region con mayor incidencia delictiva en contra de mujeres es el centro norte, esto se debe a que aqui esta la ciudad capital y otras ciudades importantes grandes.

In [919]:
totales_anio = Incidencia_Delictiva_Mujeres_Tipo_Delito_df.groupby(['anio'])['numero_delitos'].sum().reset_index()
totales_anio["porcentaje_respecto_total"] = totales_anio['numero_delitos'] / total_delitos
totales_anio = totales_anio.sort_values(by = 'porcentaje_respecto_total', ascending=False)

print(totales_anio)

    anio  numero_delitos  porcentaje_respecto_total
8   2021          8793.0                   0.141672
6   2019          7839.0                   0.126301
4   2017          7473.0                   0.120404
5   2018          7424.0                   0.119615
7   2020          6681.0                   0.107643
9   2022          6425.0                   0.103519
3   2016          4599.0                   0.074099
10  2023          3897.0                   0.062788
1   2014          3124.0                   0.050334
0   2013          3031.0                   0.048835
2   2015          2792.0                   0.044984


> El año con mayor indice delictivo sin duda alguna el 2021, un año crucial pues fue cuando las restricciones de la pandemia empezarón a quitarse, algo que bajo al año siguiente.

In [920]:
totales_mes = Incidencia_Delictiva_Mujeres_Tipo_Delito_df.groupby(['mes'])['numero_delitos'].sum().reset_index()
totales_mes["porcentaje_respecto_total"] = totales_mes['numero_delitos'] / total_delitos
totales_mes = totales_mes.sort_values(by = 'porcentaje_respecto_total', ascending=False)

print(totales_mes)

           mes  numero_delitos  porcentaje_respecto_total
6        Junio          6288.0                   0.101312
1       Agosto          5999.0                   0.096655
8         Mayo          5609.0                   0.090372
10     Octubre          5523.0                   0.088986
11  Septiembre          5517.0                   0.088889
5        Julio          5291.0                   0.085248
7        Marzo          5123.0                   0.082541
3        Enero          4981.0                   0.080253
0        Abril          4903.0                   0.078997
9    Noviembre          4819.0                   0.077643
4      Febrero          4541.0                   0.073164
2    Diciembre          3484.0                   0.056134


> Los meses mas conflictivos Junio y Agosto, revisaremos mas adelante como se comporta la temperatura y su relación y ver como afecta dicha temporada en el disparo de delitos.
>
> Los meses menos conflictos donde las temperaturas tienden a hacer mas bajas

Cantidad de delitos por tipo de delito

In [921]:
totales_tipo_delito = Incidencia_Delictiva_Mujeres_Tipo_Delito_df.groupby(['tipo_delito'])['numero_delitos'].sum().reset_index()
totales_tipo_delito["porcentaje_respecto_total"] = totales_tipo_delito['numero_delitos'] / total_delitos
totales_tipo_delito = totales_tipo_delito.sort_values(by = 'porcentaje_respecto_total', ascending=False)

print(totales_tipo_delito)

                          tipo_delito  numero_delitos  \
29                 Violencia Familiar         38280.0   
18   Incumpl. obligaciones familiares          7622.0   
21                        Negligencia          4002.0   
22                Omisión de Cuidados          2895.0   
3                   Abuso Psicológico          2055.0   
2                        Abuso Físico          1442.0   
28                          Violación          1255.0   
5                  Abusos Deshonestos           838.0   
4                        Abuso Sexual           636.0   
20     Necesidad de Asistencia social           466.0   
16                   Homicidio Doloso           440.0   
19  Mujeres Desaparecidas/Extraviadas           434.0   
11                     Discriminación           401.0   
15                        Feminicidio           257.0   
17               Hostigamiento Sexual           202.0   
0                            Abandono           189.0   
13                            E

> Violencia familiar es el de mayor incidencia, y como segundo lugar Incumplimiento obligaciones familiares, todo lo relacionado con lo familiar, esto puede ser por que pueda ser mas accesible de obtener registros por medio de las demandas o reportes.

Revisando un tipo de delito para ver como esta distribuido el numero de delitos en los meses de enero a diciembre en todos los años

In [922]:
Incidencia_Delictiva_Mujeres_Tipo_Delito_pivot_anio_df[Incidencia_Delictiva_Mujeres_Tipo_Delito_pivot_anio_df['tipo_delito'] == 'Violencia Familiar']

,mes,tipo_delito,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
29,Abril,Violencia Familiar,163.0,181.0,142.0,160.0,339.0,265.0,233.0,234.0,490.0,330.0,321.0
60,Agosto,Violencia Familiar,156.0,147.0,95.0,255.0,454.0,382.0,614.0,483.0,588.0,321.0,277.0
91,Diciembre,Violencia Familiar,79.0,122.0,112.0,221.0,164.0,204.0,285.0,407.0,284.0,202.0,0.0
122,Enero,Violencia Familiar,167.0,168.0,167.0,130.0,366.0,286.0,217.0,369.0,501.0,337.0,294.0
153,Febrero,Violencia Familiar,168.0,182.0,134.0,120.0,322.0,285.0,197.0,399.0,436.0,317.0,265.0
184,Julio,Violencia Familiar,103.0,160.0,100.0,238.0,376.0,435.0,494.0,397.0,606.0,305.0,288.0
215,Junio,Violencia Familiar,164.0,220.0,168.0,249.0,408.0,534.0,467.0,362.0,667.0,382.0,284.0
246,Marzo,Violencia Familiar,169.0,172.0,168.0,131.0,392.0,308.0,279.0,284.0,531.0,344.0,349.0
277,Mayo,Violencia Familiar,178.0,194.0,159.0,164.0,385.0,386.0,390.0,264.0,637.0,359.0,312.0
308,Noviembre,Violencia Familiar,126.0,121.0,91.0,229.0,346.0,339.0,486.0,570.0,398.0,267.0,0.0


### Otras estadisticas para seguir revisando la información

In [923]:
print(Incidencia_Delictiva_Mujeres_Tipo_Delito_pivot_mes_df.groupby('tipo_delito').agg(["mean", "std"]).fillna(0))

                                          anio                 Enero  \
                                          mean       std        mean   
tipo_delito                                                            
Abandono                           2018.000000  3.316625    1.363636   
Abuso Fetal                        2018.000000  3.316625    1.363636   
Abuso Físico                       2018.000000  3.316625   10.545455   
Abuso Psicológico                  2018.000000  3.316625   12.818182   
Abuso Sexual                       2018.000000  3.316625    4.090909   
Abusos Deshonestos                 2018.000000  3.316625    6.818182   
Acoso escolar(Bullying)            2019.142857  2.410295    0.285714   
Acoso laboral                      2019.500000  2.449490    1.000000   
Alcoholismo                        2022.000000  0.000000    0.000000   
Corrupción de Menores              2017.800000  3.425395    0.300000   
Cutting(Autolesión)                2017.000000  0.000000    0.00

In [924]:
region_2013_summary = Incidencia_Delictiva_Mujeres_Municipio_pivot_anio_df.groupby("region").agg(
    # Creando la media para el año 2013
    mean_rate_2013 = (2013, "mean"),
    # Creando la desviación estandar para el año 2013
    std_rate_2013 = (2013, "std")
)
print(region_2013_summary)

               mean_rate_2013  std_rate_2013
region                                      
Centro Norte        16.113636      45.061730
Noraeste             2.053030       3.630478
Río Sonora           0.295455       0.914169
Sierra Alta          0.427778       1.606657
Sierra Centro        0.060606       0.239515
Sur                  3.243590       5.250342


In [925]:
region_2022_summary = Incidencia_Delictiva_Mujeres_Municipio_pivot_anio_df.groupby("region").agg(
    # Creando la media para el año 2021
    mean_rate_2022 = (2022, "mean"),
    # Creando la desviación estandar para el año 2021
    std_rate_2022 = (2022, "std")
)
print(region_2022_summary)

               mean_rate_2022  std_rate_2022
region                                      
Centro Norte        28.909091      76.353682
Noraeste             4.765152       8.890221
Río Sonora           1.401515       3.737289
Sierra Alta          1.594444       4.995749
Sierra Centro        0.090909       0.359273
Sur                  9.583333      16.424591


In [926]:
region_2023_summary = Incidencia_Delictiva_Mujeres_Municipio_pivot_anio_df.groupby("region").agg(
    # Creando la media para el año 2023
    mean_rate_2023 = (2023, "mean"),
    # Creando la desviación estandar para el año 2023
    std_rate_2023 = (2023, "std")
)
print(region_2023_summary)

               mean_rate_2023  std_rate_2023
region                                      
Centro Norte        23.409091      60.170378
Noraeste             6.409091      11.818198
Río Sonora           1.454545       4.002611
Sierra Alta          0.800000       2.475425
Sierra Centro        0.113636       0.440267
Sur                  9.971154      18.553233


In [927]:
# Filtrando los datos para excluir las filas donde la cantidad de incidencias es igual a cero
df_delitos_filtrado = Incidencia_Delictiva_Mujeres_Tipo_Delito_df

df_delitos_filtrado = df_delitos_filtrado[df_delitos_filtrado['numero_delitos'] > 0]

# Agrupando los datos por año y calcula la cantidad máxima y mínima de delitos
df_delitos_filtrado_anio_df = df_delitos_filtrado.groupby('anio')['numero_delitos'].agg(['max', 'min', 'mean', 'std']).reset_index()

# Renombrando las columnas para mayor claridad
df_delitos_filtrado_anio_df = df_delitos_filtrado_anio_df.rename(columns={'max': 'max_delitos', 'min': 'min_delitos', 'mean': 'mean_delitos', 'std': 'std_delitos'})

print(df_delitos_filtrado_anio_df)

    anio  max_delitos  min_delitos  mean_delitos  std_delitos
0   2013        178.0          1.0     22.451852    41.453034
1   2014        220.0          1.0     22.314286    47.898587
2   2015        168.0          1.0     17.670886    35.258182
3   2016        347.0          1.0     23.584615    53.823277
4   2017        454.0          1.0     33.066372    83.877742
5   2018        534.0          1.0     32.561404    81.628258
6   2019        614.0          1.0     35.958716    96.615505
7   2020        570.0          1.0     33.074257    95.643562
8   2021        667.0          1.0     45.796875   124.342650
9   2022        382.0          1.0     30.023364    72.890389
10  2023        349.0          1.0     29.300752    70.534860


In [928]:
# Filtrando los datos para excluir las filas donde la cantidad de incidencias es igual a cero
df_delitos_filtrado = Incidencia_Delictiva_Mujeres_Tipo_Delito_df

# Agrupando los datos por año y calcula la cantidad máxima y mínima de delitos
df_delitos_filtrado_delito_df = df_delitos_filtrado.groupby('tipo_delito')['numero_delitos'].agg(['max', 'min', 'mean', 'std']).reset_index()

# Renombrando las columnas para mayor claridad
df_delitos_filtrado_delito_df = df_delitos_filtrado_delito_df.rename(columns={'max': 'max_delitos', 'min': 'min_delitos', 'mean': 'mean_delitos', 'std': 'std_delitos'})

print(df_delitos_filtrado_delito_df)

                          tipo_delito  max_delitos  min_delitos  mean_delitos  \
0                            Abandono         10.0          0.0      1.476562   
1                         Abuso Fetal          6.0          0.0      0.812500   
2                        Abuso Físico         45.0          0.0     11.265625   
3                   Abuso Psicológico         70.0          0.0     16.054688   
4                        Abuso Sexual         15.0          0.0      4.968750   
5                  Abusos Deshonestos         30.0          0.0      6.546875   
6             Acoso escolar(Bullying)          4.0          0.0      0.537500   
7                       Acoso laboral          5.0          0.0      0.902174   
8                         Alcoholismo          2.0          0.0      0.250000   
9               Corrupción de Menores          5.0          0.0      0.543103   
10                Cutting(Autolesión)          1.0          0.0      0.083333   
11                     Discr

In [929]:
# Filtrando los datos para excluir las filas donde la cantidad de incidencias es igual a cero
df_delitos_filtrado = Incidencia_Delictiva_Mujeres_Tipo_Delito_df

# Agrupando los datos por año y calcula la cantidad máxima y mínima de delitos
df_delitos_filtrado_mes_df = df_delitos_filtrado.groupby('mes')['numero_delitos'].agg(['max', 'min', 'mean', 'std']).reset_index()

# Renombrando las columnas para mayor claridad
df_delitos_filtrado_mes_df = df_delitos_filtrado_mes_df.rename(columns={'max': 'max_delitos', 'min': 'min_delitos', 'mean': 'mean_delitos', 'std': 'std_delitos'})

df_delitos_filtrado_mes_df['mes'] = pd.Categorical(df_delitos_filtrado_mes_df['mes'], categories=orden_meses, ordered=True)
df_delitos_filtrado_mes_df = df_delitos_filtrado_mes_df.sort_values(by='mes')
df_delitos_filtrado_mes_df = df_delitos_filtrado_mes_df.reset_index(drop=True)

print(df_delitos_filtrado_mes_df)

           mes  max_delitos  min_delitos  mean_delitos  std_delitos
0        Enero        501.0          0.0     19.610236    60.819386
1      Febrero        436.0          0.0     17.877953    56.794937
2        Marzo        531.0          0.0     20.169291    63.022730
3        Abril        490.0          0.0     19.303150    57.524343
4         Mayo        637.0          0.0     22.082677    70.274146
5        Junio        667.0          0.0     24.755906    79.703275
6        Julio        606.0          0.0     20.830709    72.983179
7       Agosto        614.0          0.0     23.618110    79.402162
8   Septiembre        532.0          0.0     23.780172    76.439201
9      Octubre        548.0          0.0     23.806034    75.760338
10   Noviembre        570.0          0.0     20.771552    68.943383
11   Diciembre        407.0          0.0     15.017241    47.188025


In [930]:
# Filtrando los datos para excluir las filas donde la cantidad de incidencias es igual a cero
df_delitos_filtrado = Incidencia_Delictiva_Mujeres_Municipio_df

# Agrupando los datos por año y calcula la cantidad máxima y mínima de delitos
df_delitos_filtrado_region_df = df_delitos_filtrado.groupby('region')['numero_delitos'].agg(['max', 'min', 'mean', 'std']).reset_index()

# Renombrando las columnas para mayor claridad
df_delitos_filtrado_region_df = df_delitos_filtrado_region_df.rename(columns={'max': 'max_delitos', 'min': 'min_delitos', 'mean': 'mean_delitos', 'std': 'std_delitos'})

print(df_delitos_filtrado_region_df)

          region  max_delitos  min_delitos  mean_delitos  std_delitos
0   Centro Norte        416.0          0.0     24.658381    66.374038
1       Noraeste         92.0          0.0      4.556818    10.608074
2     Río Sonora         21.0          0.0      0.834517     2.333655
3    Sierra Alta         31.0          0.0      0.948958     3.124063
4  Sierra Centro         17.0          0.0      0.225852     0.986828
5            Sur        207.0          0.0     10.586538    23.905551


Creación de datafranes que quitan los datos de 0 delitos cometidos en contra de mujeres
> Esto para poder usarlos en el siguiente analisis y ver el comportamiento de los datos sin incluir la incidencia 0.

In [931]:
sin_cero_Df = Incidencia_Delictiva_Mujeres_Municipio_df[Incidencia_Delictiva_Mujeres_Municipio_df['numero_delitos'] > 0]

sin_cero_tipo_Df = Incidencia_Delictiva_Mujeres_Tipo_Delito_df[Incidencia_Delictiva_Mujeres_Tipo_Delito_df['numero_delitos'] > 0]

print(np.quantile(sin_cero_Df['numero_delitos'], [0, 0.25, 0.5, 0.75, 1]))

[  1.   1.   3.  12. 416.]


Creando series cada uno una lista diferente (Región y Municipio), los cuales apoyaran en la generación de graficos.

In [932]:
lista_regiones = municipios_regiones['Region'].unique()

print(lista_regiones)

lista_municipios = municipios_regiones['municipio'].unique()
print(lista_municipios)

['Noraeste' 'Centro Norte' 'Sur' 'Río Sonora' 'Sierra Alta'
 'Sierra Centro']
['Altar' 'Átil' 'Caborca' 'Oquitoa' 'Pitiquito'
 'General Plutarco Elías Calles' 'Puerto Peñasco' 'San Luis Río Colorado'
 'Sáric' 'Trincheras' 'Tubutama' 'Benjamín Hill' 'Carbó' 'Cucurpe'
 'Hermosillo' 'Ímuris' 'Magdalena' 'Nogales' 'Santa Ana' 'Santa Cruz'
 'Opodepe' 'San Miguel de Horcasitas' 'Álamos' 'Bácum' 'Benito Juárez'
 'Cajeme' 'Empalme' 'Etchojoa' 'Guaymas' 'Huatabampo' 'Navojoa' 'Quiriego'
 'Rosario' 'San Ignacio Río Muerto' 'Yécora' 'Aconchi' 'Arizpe' 'Bacoachi'
 'Banámichi' 'Baviácora' 'Cananea' 'Huépac' 'Ures' 'Naco' 'Rayón'
 'San Felipe de Jesús' 'Agua Prieta' 'Bacadéhuachi' 'Bacerac' 'Bavispe'
 'Cumpas' 'Divisaderos' 'Fronteras' 'Granados' 'Huachinera' 'Huásabas'
 'Moctezuma' 'Nácori Chico' 'Nacozari de García' 'Tepache' 'Villa Hidalgo'
 'Arivechi' 'Bacanora' 'La Colorada' 'Mazatán' 'Ónavas' 'Sahuaripa'
 'San Javier' 'San Pedro de la Cueva' 'Soyopa' 'Suaqui Grande'
 'Villa Pesqueira']


#### Generación de columnas nuevas para un  mejor entendimiento de los datos (control de la dispersión)

In [933]:
#Agrupando por año obtener la suma de los numeros de delitos cometido en un determinado año
numero_anio = sin_cero_tipo_Df.groupby(['anio'])['numero_delitos'].agg('sum').reset_index()
#Se renombra para cambiar el nombre de la columna por dataframe anterior
numero_anio = numero_anio.rename(columns={'numero_delitos':'total_delitos_anio'})
delitos_tipo_anio = sin_cero_tipo_Df.merge(numero_anio, how='inner', on=['anio'])
delitos_tipo_anio['numero_delitos_anio_rel'] = delitos_tipo_anio['numero_delitos']/delitos_tipo_anio['total_delitos_anio']

#Agrupando por año obtener la suma de los numeros de delitos cometido en un determinado año
numero_municipio_anio = sin_cero_Df.groupby(['anio'])['numero_delitos'].agg('sum').reset_index()
#Se renombra para cambiar el nombre de la columna por dataframe anterior
numero_municipio_anio = numero_municipio_anio.rename(columns={'numero_delitos':'total_delitos_anio'})

delitos_municipio_anio = sin_cero_Df.merge(numero_municipio_anio, how='inner', on=['anio'])
delitos_municipio_anio['numero_delitos_anio_rel'] = delitos_municipio_anio['numero_delitos']/delitos_municipio_anio['total_delitos_anio']


#Agrupando por año y mes para obtener la suma de los numeros de delitos cometido en un determinado mes de un año
numero_anio_mes = sin_cero_Df.groupby(['anio', 'mes'])['numero_delitos'].agg('sum').reset_index()
#Se renombra para cambiar el nombre de la columna por dataframe anterior
numero_anio_mes = numero_anio_mes.rename(columns={'numero_delitos':'total_delitos_anio_mes'})
delitos_municipio_porcentaje = sin_cero_Df.merge(numero_anio_mes, how='inner', on=['anio', 'mes'])
delitos_municipio_porcentaje['numero_delitos_anio_mes_rel'] = delitos_municipio_porcentaje['numero_delitos']/delitos_municipio_porcentaje['total_delitos_anio_mes']

#Agrupando por region para obtener la suma de los numeros de delitos cometido en una determinada región
numero_region = sin_cero_Df.groupby(['region'])['numero_delitos'].agg('sum').reset_index()
numero_region = numero_region.rename(columns={'numero_delitos':'total_delitos_region'})
delitos_municipio_porcentaje = delitos_municipio_porcentaje.merge(numero_region, how='inner', on=['region'])
delitos_municipio_porcentaje['numero_delitos_region_rel'] = delitos_municipio_porcentaje['numero_delitos']/delitos_municipio_porcentaje['total_delitos_region']

print(delitos_municipio_porcentaje)


      anio        mes         region      municipio  numero_delitos  \
0     2013      Enero   Centro Norte     Hermosillo           162.0   
1     2013      Enero   Centro Norte        Nogales            28.0   
2     2013      Enero   Centro Norte      Magdalena             6.0   
3     2013      Enero   Centro Norte  Benjamín Hill             1.0   
4     2014      Enero   Centro Norte     Hermosillo           163.0   
...    ...        ...            ...            ...             ...   
3313  2018  Diciembre  Sierra Centro  Suaqui Grande             1.0   
3314  2019  Diciembre  Sierra Centro      Sahuaripa             4.0   
3315  2020  Diciembre  Sierra Centro      Sahuaripa             2.0   
3316  2020  Diciembre  Sierra Centro        Mazatán             1.0   
3317  2021  Diciembre  Sierra Centro     San Javier             1.0   

      total_delitos_anio_mes  numero_delitos_anio_mes_rel  \
0                      284.0                     0.570423   
1                      28

In [934]:
print(numero_anio)
print(delitos_municipio_anio)
print(delitos_tipo_anio)

    anio  total_delitos_anio
0   2013              3031.0
1   2014              3124.0
2   2015              2792.0
3   2016              4599.0
4   2017              7473.0
5   2018              7424.0
6   2019              7839.0
7   2020              6681.0
8   2021              8793.0
9   2022              6425.0
10  2023              3897.0
      anio     mes         region                 municipio  numero_delitos  \
0     2013   Enero   Centro Norte                Hermosillo           162.0   
1     2013   Enero   Centro Norte                   Nogales            28.0   
2     2013   Enero            Sur                    Cajeme            25.0   
3     2013   Enero       Noraeste     San Luis Río Colorado            10.0   
4     2013   Enero            Sur                   Navojoa            11.0   
...    ...     ...            ...                       ...             ...   
3313  2023  Agosto    Sierra Alta                 Moctezuma             1.0   
3314  2023  Agosto  

In [935]:
numero_tipo_delitos = sin_cero_tipo_Df.groupby(['tipo_delito'])['numero_delitos'].agg('sum').reset_index()
numero_tipo_delitos = numero_tipo_delitos.rename(columns={'numero_delitos':'total_delitos'})

delitos_tipo_porcentaje = sin_cero_tipo_Df.merge(numero_tipo_delitos, how='inner', on=['tipo_delito'])

delitos_tipo_porcentaje['numero_delitos_rel'] = delitos_tipo_porcentaje['numero_delitos']/delitos_tipo_porcentaje['total_delitos']

print(delitos_tipo_porcentaje)

      anio         mes         tipo_delito  numero_delitos  total_delitos  \
0     2013       Enero  Violencia Familiar           167.0        38280.0   
1     2014       Enero  Violencia Familiar           168.0        38280.0   
2     2015       Enero  Violencia Familiar           167.0        38280.0   
3     2016       Enero  Violencia Familiar           130.0        38280.0   
4     2017       Enero  Violencia Familiar           366.0        38280.0   
...    ...         ...                 ...             ...            ...   
2036  2019       Julio   Trata de personas             3.0            6.0   
2037  2021      Agosto   Trata de personas             1.0            6.0   
2038  2018   Noviembre   Trata de personas             1.0            6.0   
2039  2022      Agosto         Alcoholismo             1.0            3.0   
2040  2022  Septiembre         Alcoholismo             2.0            3.0   

      numero_delitos_rel  
0               0.004363  
1               0.004

### BOXPLOTs para analisis del numero de delitos y su distribución

Boxplot para analizar por año como se distribuye los diferentes delitos en cada uno de las regiones

In [936]:
# Calculando el número de filas y columnas para el diseño de subgráficos
n_rows = (len(lista_anio) + 1) // 2
n_cols = 2

# Creando un contenedor para los gráficos de barras
fig = sp.make_subplots(rows=n_rows, cols=n_cols, subplot_titles=[f'Año {year}' for year in lista_anio])

# Ajustando el espacio entre subgráficos
fig.update_layout(height=500 * n_rows, showlegend=False)

# Iterando a través de los años y agregar un boxplot por año
for i, anio in enumerate(lista_anio):
    row, col = i // n_cols + 1, i % n_cols + 1  # +1 para evitar la celda superior izquierda que contiene los títulos

    sin_cero_anio_df = sin_cero_Df[sin_cero_Df['anio'] == anio]

    # Creando boxplot
    fig.add_trace(px.box(sin_cero_anio_df, x="region", y="numero_delitos").data[0], row=row, col=col)
    
    # Personalizando etiquetas y título
    fig.update_xaxes(title_text="Región", row=row, col=col)
    fig.update_yaxes(title_text="Número de Delitos relación", row=row, col=col)
    #box_fig.update_xaxes(categoryorder="total descending")
    #box_fig.update_layout(xaxis=dict(tickmode="array", tickvals=list(range(len(lista_regiones))), ticktext=lista_regiones))
    fig.update_layout(title=f'Distribución de Delitos por Región - Año {anio}')

fig.show()

#### Boxplot de distribución de a incidencia en las distintas regiones para el primer año obtenido (2013)

In [937]:
sin_cero_anio_Df = sin_cero_Df[sin_cero_Df['anio']==2013]
 
# Creando boxplot a partir de un dataframe que descarta la incidencia delictica en 0
fig = px.box(sin_cero_anio_Df, x="region", y="numero_delitos", color="region")

# Personalizando las etiquetas de los ejes x e y
fig.update_xaxes(title_text="Región")
fig.update_yaxes(title_text="Número de Delitos relación")

# Personalizando las leyendas en el eje x
fig.update_xaxes(categoryorder="total descending")  # Ordenando las categorías de x por la suma de "numero_delitos"
fig.update_layout(xaxis=dict(tickmode="array", tickvals=list(range(len(lista_regiones))), ticktext=lista_regiones))

# Agregando un título al gráfico
fig.update_layout(title="Distribución de Delitos por Región - Año 2013")

# Otros ajustes visuales
fig.update_traces(boxpoints="all", jitter=0.3, pointpos=-2)  # Mostrando todos los puntos y agregar jitter
#fig.update_layout(showlegend=False)  # Ocultando la leyenda si no es necesaria

fig.show()

#### Boxplot de distribución en las distintas regiones para el año en e que se tiene el mayor numero de incidencia (2021)

In [938]:
sin_cero_anio_Df = sin_cero_Df[sin_cero_Df['anio']==2021]
 
# Creando boxplot a partir de un dataframe que descarta la incidencia delictica en 0
fig = px.box(sin_cero_anio_Df, x="region", y="numero_delitos", color="region")

# Personalizando las etiquetas de los ejes x e y
fig.update_xaxes(title_text="Región")
fig.update_yaxes(title_text="Número de Delitos relación")

# Personalizando las leyendas en el eje x
fig.update_xaxes(categoryorder="total descending")  # Ordenando las categorías de x por la suma de "numero_delitos"
fig.update_layout(xaxis=dict(tickmode="array", tickvals=list(range(len(lista_regiones))), ticktext=lista_regiones))

# Agregando un título al gráfico
fig.update_layout(title="Distribución de Delitos por Región - Año 2021")

# Otros ajustes visuales
fig.update_traces(boxpoints="all", jitter=0.3, pointpos=-2)  # Mostrando todos los puntos y agregar jitter
#fig.update_layout(showlegend=False)  # Ocultando la leyenda si no es necesaria

fig.show()

#### Boxplot de distribución de a incidencia en las distintas regiones para el año anterior al actual (2022)
> Este año se toma y no el actual debido a que es el ultimo año que tiene los datos completos

In [939]:
sin_cero_anio_Df = sin_cero_Df[sin_cero_Df['anio']==2022]
 
# Creando boxplot a partir de un dataframe que descarta la incidencia delictica en 0
fig = px.box(sin_cero_anio_Df, x="region", y="numero_delitos", color="region")

# Personalizando las etiquetas de los ejes x e y
fig.update_xaxes(title_text="Región")
fig.update_yaxes(title_text="Número de Delitos relación")

# Personalizando las leyendas en el eje x
fig.update_xaxes(categoryorder="total descending")  # Ordenando las categorías de x por la suma de "numero_delitos"
fig.update_layout(xaxis=dict(tickmode="array", tickvals=list(range(len(lista_regiones))), ticktext=lista_regiones))

# Agregando un título al gráfico
fig.update_layout(title="Distribución de Delitos por Región - Año 2022")

# Otros ajustes visuales
fig.update_traces(boxpoints="all", jitter=0.3, pointpos=-2)  # Mostrando todos los puntos y agregar jitter
#fig.update_layout(showlegend=False)  # Ocultando la leyenda si no es necesaria

fig.show()

### Detectando Outlier

Municipios Outliers

In [940]:
# Calculando el numero de delitos por municipio
delitos_municipio = Incidencia_Delictiva_Mujeres_Municipio_df.groupby('municipio')['numero_delitos'].sum()

# Generando los quartiles para poder obtemer mas adelante los outlier
q1 = np.quantile(delitos_municipio, 0.25)
q3 = np.quantile(delitos_municipio, 0.75)
iqr = q3 - q1

# Calculate the lower and upper cutoffs for outliers
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

# Subconjunto de delitos por municipio para encontrar valores tipicos
outliers = delitos_municipio[(delitos_municipio < lower) | (delitos_municipio > upper) ]
print(outliers)

municipio
Agua Prieta               1318.0
Caborca                   1814.0
Cajeme                    8768.0
Cananea                    758.0
Etchojoa                  1058.0
Guaymas                   1913.0
Hermosillo               27755.0
Huatabampo                 961.0
Magdalena                  408.0
Navojoa                   3610.0
Nogales                   6013.0
Puerto Peñasco             887.0
San Luis Río Colorado     3460.0
Name: numero_delitos, dtype: float64


Tipos de delitos Outliers

In [941]:
# Calculando el numero de delitos por tipo de delito
delitos_tipo = Incidencia_Delictiva_Mujeres_Tipo_Delito_Sin_0_df.groupby('tipo_delito')['numero_delitos'].sum()

# Generando los quartiles para poder obtemer mas adelante los outlier
q1 = np.quantile(delitos_tipo, 0.25)
q3 = np.quantile(delitos_tipo, 0.75)
iqr = q3 - q1

# Calculate the lower and upper cutoffs for outliers
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

# Subconjunto de delitos por tipo de delito para encontrar valores tipicos
outliers = delitos_tipo[(delitos_tipo < lower) | (delitos_tipo > upper) ]
print(outliers)

tipo_delito
Abuso Psicológico                    2055.0
Incumpl. obligaciones familiares     7622.0
Negligencia                          4002.0
Omisión de Cuidados                  2895.0
Violencia Familiar                  38280.0
Name: numero_delitos, dtype: float64


## Graficas de Exploración

In [942]:
# Calculando el número de filas y columnas para el diseño de subgráficos
n_rows = (len(lista_anio) + 1) // 2
n_cols = 2

# Creando un contenedor para los gráficos de barras
fig = sp.make_subplots(rows=n_rows, cols=n_cols, subplot_titles=[f'Año {year}' for year in lista_anio])

# Ajustando el espacio entre subgráficos
fig.update_layout(height=400 * n_rows, showlegend=False)

# Iteracion a través de los años y crea un gráfico de barras para cada uno
for i, year in enumerate(lista_anio):
    row, col = i // n_cols + 1, i % n_cols + 1  # +1 para evitar la celda superior izquierda que contiene los títulos

    #data = Incidencia_Delictiva_Mujeres_Municipio_pivot_anio_df.sort_values(by=year, ascending=False)
    data = Incidencia_Delictiva_Mujeres_Municipio_pivot_anio_df.groupby('region')[year].sum().reset_index()
    data = data.sort_values(by=year, ascending=True)
    title = f'Año {year}'

    # Creando un gráfico de barras
    fig.add_trace(px.bar(data, x=year, y="region", title=title, text=year ,orientation="h").data[0], row=row, col=col)

    # Ajustando las etiquetas de los ejes
    #fig.update_xaxes(title_text="Número de Delitos", row=n_rows // 2 + 1, col=1)
    #fig.update_yaxes(title_text="Región", row=n_rows, col=n_cols // 2 + 1)
    fig.update_xaxes(title_text="Número de Delitos", row=row, col=col)
    fig.update_yaxes(title_text="Región", row=row, col=col)

    fig.update_traces(texttemplate='%{text}', textposition='outside')

fig.show()

In [943]:
# Agrupando los datos por año y región y suma el número de delitos
df_grouped = Incidencia_Delictiva_Mujeres_Municipio_df.groupby(['anio', 'region'])['numero_delitos'].sum().reset_index()

# Creando un gráfico de barras apiladas
fig = go.Figure()

for region in df_grouped['region'].unique():
    region_data = df_grouped[df_grouped['region'] == region]
    region_delitos = [region_data[region_data['anio'] == year]['numero_delitos'].values[0] if year in region_data['anio'].values else 0 for year in lista_anio]
    fig.add_trace(go.Bar(x=lista_anio, y=region_delitos, name=region))

# Personalizando las etiquetas de los ejes
fig.update_xaxes(title_text='Año', tickvals=lista_anio, ticktext=[str(year) for year in lista_anio])
fig.update_yaxes(title_text='Número de Delitos')

fig.update_layout(title='Incidencia Delictiva contra Mujeres por Año y Región')
fig.update_layout(legend_title_text='Región')

fig.show()

In [944]:
# Agrupando los datos por mes y región y suma el número de delitos
df_grouped = Incidencia_Delictiva_Mujeres_Municipio_df.groupby(['mes', 'region'])['numero_delitos'].sum().reset_index()

# Creando un gráfico de barras apiladas
fig = go.Figure()

for region in df_grouped['region'].unique():
    region_data = df_grouped[df_grouped['region'] == region]
    region_delitos = [region_data[region_data['mes'] == mes]['numero_delitos'].values[0] if mes in region_data['mes'].values else 0 for mes in lista_mes]
    fig.add_trace(go.Bar(x=lista_mes, y=region_delitos, name=region))

# Personalizando las etiquetas de los ejes
fig.update_xaxes(title_text='Mes', tickvals=lista_mes, ticktext=[str(mes) for mes in lista_mes])
fig.update_yaxes(title_text='Número de Delitos')

fig.update_layout(title='Incidencia Delictiva contra Mujeres por Mes y Región')
fig.update_layout(legend_title_text='Región')

fig.show()

### Comparación Incidencia Delictiva vs Temperatura
> En la sección siguiente se comparará los datos que se tienen registrado de la temperatura en los distintos municipios del estado, contra los datos de la incidencia delictiva en contra de mujeres para poder encontrar alguna relación y ver el comportamiento de los datos.

In [945]:
# Merge con la tabla de clima con la incidencia delictiva contra mujeres con datos de tipo de delitos
mujer_temperatura_merge_df = pd.merge(Incidencia_Delictiva_Mujeres_Tipo_Delito_df, clima_estatal_df, on=['anio', 'mes'], how='inner')
mujer_temperatura_merge_df['horas_luz'] = mujer_temperatura_merge_df['minutos_luz'] / 60

# Merge con la tabla de clima con la incidencia delictiva contra mujeres con datos de municipio

Incidencia_Delictiva_Mujeres_Municipio_df['municipio_normalizada'] = Incidencia_Delictiva_Mujeres_Municipio_df['municipio'].apply(unidecode)
clima_municipal_df['municipio_normalizada'] = clima_municipal_df['municipio'].apply(unidecode)
Incidencia_Delictiva_Mujeres_Municipio_df['municipio_normalizada'] = Incidencia_Delictiva_Mujeres_Municipio_df['municipio_normalizada'].str.lower()
clima_municipal_df['municipio_normalizada'] = clima_municipal_df['municipio_normalizada'].str.lower()

mujer_temperatura_region_merge_df = pd.merge(Incidencia_Delictiva_Mujeres_Municipio_df, clima_municipal_df, on=['anio', 'mes', 'municipio_normalizada'], how='inner')

mujer_temperatura_region_merge_df.drop(['municipio_normalizada', 'municipio_x'], axis=1, inplace=True)
mujer_temperatura_region_merge_df.drop(['region_y'], axis=1, inplace=True)
mujer_temperatura_region_merge_df.rename(columns = {'municipio_y': 'municipio'}, inplace = True)
mujer_temperatura_region_merge_df.rename(columns = {'region_x': 'region'}, inplace = True)

mujer_temperatura_region_merge_df.insert(3, 'municipio', mujer_temperatura_region_merge_df.pop('municipio'))


In [946]:
# Creando un gráfico de dispersión
fig = px.scatter(mujer_temperatura_merge_df, x='temperatura_promedio', y='numero_delitos', color='tipo_delito')

# Personalizando las etiquetas de los ejes
fig.update_xaxes(title_text='Temperatura Promedio')
fig.update_yaxes(title_text='Cantidad de Incidencias')

fig.update_layout(title='Relación entre Temperatura y Incidencias de Delitos')
fig.update_layout(legend_title_text='Tipo de Delito', legend=dict(x=1, y=1))

fig.show()

In [947]:
#print(numero_anio)
#print(delitos_municipio_anio)
#print(delitos_tipo_anio)

# Merge con la tabla de clima con la incidencia delictiva contra mujeres con datos de tipo de delitos
mujer_temperatura_merge_relativa_df = pd.merge(delitos_tipo_anio, clima_estatal_df, on=['anio', 'mes'], how='inner')
mujer_temperatura_merge_relativa_df['horas_luz'] = mujer_temperatura_merge_relativa_df['minutos_luz'] / 60

# Merge con la tabla de clima con la incidencia delictiva contra mujeres con datos de municipio

delitos_municipio_anio['municipio_normalizada'] = delitos_municipio_anio['municipio'].apply(unidecode)
clima_municipal_df['municipio_normalizada'] = clima_municipal_df['municipio'].apply(unidecode)
delitos_municipio_anio['municipio_normalizada'] = delitos_municipio_anio['municipio_normalizada'].str.lower()
clima_municipal_df['municipio_normalizada'] = clima_municipal_df['municipio_normalizada'].str.lower()

mujer_temperatura_region_merge_relativa_df = pd.merge(delitos_municipio_anio, clima_municipal_df, on=['anio', 'mes', 'municipio_normalizada'], how='inner')

mujer_temperatura_region_merge_relativa_df.drop(['municipio_normalizada', 'municipio_x'], axis=1, inplace=True)
mujer_temperatura_region_merge_relativa_df.drop(['region_y'], axis=1, inplace=True)
mujer_temperatura_region_merge_relativa_df.rename(columns = {'municipio_y': 'municipio'}, inplace = True)
mujer_temperatura_region_merge_relativa_df.rename(columns = {'region_x': 'region'}, inplace = True)

mujer_temperatura_region_merge_relativa_df.insert(3, 'municipio', mujer_temperatura_region_merge_relativa_df.pop('municipio'))


In [948]:
# Creando un gráfico de dispersión
fig = px.scatter(mujer_temperatura_merge_relativa_df, x='temperatura_promedio', y='numero_delitos_anio_rel', color='tipo_delito')

# Personalizando las etiquetas de los ejes
fig.update_xaxes(title_text='Temperatura Promedio')
fig.update_yaxes(title_text='Cantidad de Incidencias')

fig.update_layout(title='Relación entre Temperatura y Incidencias de Delitos')
fig.update_layout(legend_title_text='Tipo de Delito', legend=dict(x=1, y=1))

fig.show()

In [949]:
# Creando un gráfico de dispersión
fig = px.scatter(mujer_temperatura_merge_df, x='horas_luz', y='numero_delitos', color='tipo_delito')

# Personalizando las etiquetas de los ejes
fig.update_xaxes(title_text='Horas Luz de Día')
fig.update_yaxes(title_text='Cantidad de Incidencias')

fig.update_layout(title='Relación entre Horas Luz de Día y Incidencias de Delitos')
fig.update_layout(legend_title_text='Tipo de Delito', legend=dict(x=1, y=1))

fig.show()

In [950]:
# Creando un gráfico de dispersión
fig = px.scatter(mujer_temperatura_region_merge_df, x='temperatura_promedio', y='numero_delitos', color='region')

# Personalizando las etiquetas de los ejes
fig.update_xaxes(title_text='Temperatura Promedio')
fig.update_yaxes(title_text='Cantidad de Incidencias')

fig.update_layout(title='Relación entre Temperatura y Incidencias de Delitos')
fig.update_layout(legend_title_text='Región', legend=dict(x=1, y=1))

fig.show()

In [951]:
clima_municipal_df.head()

,anio,mes,entidad,region,municipio,temperatura_maxima,temperatura_minima,temperatura_promedio,salida_sol,puesta_sol,minutos_luz,precipitacion_total,lluvia_total,tiempo_precipitacion,municipio_normalizada
0,2015,Enero,Sonora,Río Sonora,Aconchi,22.37,8.91,15.05,7:15,17:44,628.65,81.7,81.7,56.0,aconchi
1,2015,Enero,Sonora,Sierra Alta,Agua Prieta,18.09,4.62,10.96,7:14,17:38,624.06,60.7,60.7,68.0,agua prieta
2,2015,Enero,Sonora,Sur,Alamos,25.37,12.31,18.26,7:05,17:44,639.42,45.0,45.0,47.0,alamos
3,2015,Enero,Sonora,Noraeste,Altar,22.10,7.84,14.94,7:24,17:49,625.16,38.1,38.1,53.0,altar
4,2015,Enero,Sonora,Sierra Centro,Arivechi,23.91,8.37,15.87,7:09,17:42,632.13,41.8,41.8,56.0,arivechi


In [952]:
Incidencia_Delictiva_Mujeres_Municipio_df['municipio'].unique()

array(['Hermosillo', 'Nogales', 'Cajeme', 'San Luis Río Colorado',
       'Navojoa', 'Caborca', 'Agua Prieta', 'Guaymas', 'Puerto Peñasco',
       'Huatabampo', 'Etchojoa', 'Cananea', 'Benito Juárez', 'Magdalena',
       'Empalme', 'Álamos', 'Altar', 'General Plutarco Elías Calles',
       'Nacozari de García', 'Benjamín Hill', 'Bácum',
       'San Miguel de Horcasitas', 'Naco', 'Ures',
       'San Ignacio Río Muerto', 'Sahuaripa', 'Santa Ana', 'Ímuris',
       'Pitiquito', 'Banámichi', 'Mazatán', 'Villa Hidalgo', 'Moctezuma',
       'Rayón', 'Villa Pesqueira', 'Rosario', 'San Javier',
       'San Pedro de la Cueva', 'Sáric', 'Yécora', 'Bacoachi',
       'Baviácora', 'Carbó', 'San Felipe de Jesús', 'Santa Cruz',
       'Tubutama', 'Aconchi', 'Átil', 'Bacerac', 'Opodepe', 'Cucurpe',
       'Granados', 'Tepache', 'Huachinera', 'Huépac', 'Arivechi',
       'Bacadéhuachi', 'Nácori Chico', 'Oquitoa', 'Quiriego', 'Cumpas',
       'Soyopa', 'Trincheras', 'Huásabas', 'Arizpe', 'Bacanora', 'Óna

In [953]:
corr = Frecuencia_Relativa_Delito_Region(Incidencia_Delictiva_Mujeres_Municipio_df, clima_municipal_df, 'Hermosillo', 'mes', 2, 0, '', '', 'box')

print(corr)

Frecuencia_Relativa_Delito_Region: unsupported operand type(s) for /: 'str' and 'int'
0


In [954]:
clima_estatal_df.head()

,anio,mes,entidad,temperatura_maxima,temperatura_minima,temperatura_promedio,salida_sol,puesta_sol,precipitacion_total,lluvia_total,tiempo_precipitacion,minutos_luz
0,2015,Enero,Sonora,21.95,8.42,14.76,7:14,17:44,3838.5,3836.7,4087.0,629.63
1,2015,Febrero,Sonora,24.58,10.21,17.09,6:59,18:08,1226.1,1226.1,1685.0,669.26
2,2015,Marzo,Sonora,26.36,12.02,19.36,6:28,18:28,2452.8,2452.8,3522.0,720.24
3,2015,Abril,Sonora,28.94,13.90,21.70,5:53,18:47,1349.0,1349.0,1442.0,773.66
4,2015,Mayo,Sonora,30.98,15.71,23.72,5:27,19:06,104.2,104.2,318.0,818.29


In [955]:
Incidencia_Delictiva_Mujeres_Tipo_Delito_df['tipo_delito'].unique()

array(['Violencia Familiar', 'Negligencia', 'Abuso Físico',
       'Incumpl. obligaciones familiares', 'Omisión de Cuidados',
       'Necesidad de Asistencia social', 'Abuso Sexual',
       'Homicidio Doloso', 'Abandono', 'Abuso Psicológico', 'Violación',
       'Abusos Deshonestos', 'Mujeres Desaparecidas/Extraviadas',
       'Corrupción de Menores', 'Drogadicción', 'Abuso Fetal',
       'Feminicidio', 'Sustracción del menor', 'Explotación Laboral',
       'Estupro', 'Pornografia infantil', 'Hostigamiento Sexual',
       'Acoso laboral', 'Acoso escolar(Bullying)', 'Discriminación',
       'Sexteo(Sexting)', 'Violencia en el noviazgo',
       'Cutting(Autolesión)', 'Trata de personas', 'Trata de Infante',
       'Alcoholismo'], dtype=object)

In [956]:
corr = Frecuencia_Relativa_Delito_tipo(Incidencia_Delictiva_Mujeres_Tipo_Delito_df, clima_municipal_df, 'Feminicidio', 'mes', 2, 0, '', '', 'box')
print(corr)

Frecuencia_Relativa_Delito_tipo: unsupported operand type(s) for /: 'str' and 'int'
0


In [957]:
# Creando un gráfico de dispersión para las variables de Delitos
fig = px.scatter_matrix(mujer_temperatura_merge_df, dimensions=["anio", "mes", "temperatura_maxima", "temperatura_minima", "temperatura_promedio", "numero_delitos"],
                        color="tipo_delito", title="Variables de Delitos")

# Personalizando el tamaño de la figura
fig.update_layout(height=900)

fig.show()

In [958]:
# Creando un gráfico de dispersión para las variables de Delitos
fig = px.scatter_matrix(mujer_temperatura_merge_df, dimensions=["anio", "mes", "salida_sol", "puesta_sol", "minutos_luz", "numero_delitos"],
                        color="tipo_delito", title="Variables de Delitos")

# Personalizando el tamaño de la figura
fig.update_layout(height=900)

fig.show()

In [959]:
# Creando un gráfico de dispersión para las variables de Delitos
fig = px.scatter_matrix(mujer_temperatura_merge_df, dimensions=["anio", "mes", "precipitacion_total", "lluvia_total", "tiempo_precipitacion", "numero_delitos"],
                        color="tipo_delito", title="Variables de Delitos")

# Personalizando el tamaño de la figura
fig.update_layout(height=900)

fig.show()

In [960]:
# Agrupando los datos por región y suma el número de delitos
df_grouped = Incidencia_Delictiva_Mujeres_Municipio_df.groupby(['region'])['numero_delitos'].sum().reset_index()
df_grouped = df_grouped.sort_values(by='numero_delitos', ascending=False)

fig = px.bar(df_grouped, x='numero_delitos', y='region', text="numero_delitos" ,orientation='h', color='region')

fig.update_xaxes(title_text='Número de Delitos')
fig.update_yaxes(title_text='Regiones')

fig.update_layout(title='Número de Delitos por Región')
fig.update_layout(height=400)

# Personalizando el texto en las barras
fig.update_traces(texttemplate='%{text}', textposition='outside')

fig.show()

In [961]:
# Agrupando los datos por municipio y suma el número de delitos
df_grouped = Incidencia_Delictiva_Mujeres_Municipio_df.groupby(['municipio'])['numero_delitos'].sum().reset_index()
df_grouped = df_grouped.sort_values(by='numero_delitos', ascending=False)

# Filtrando los 10 municipios con el número más alto de delitos
top_10_municipios = df_grouped.nlargest(10, 'numero_delitos')

fig = px.bar(top_10_municipios, x='numero_delitos', y='municipio', text="numero_delitos" ,orientation='h', color='municipio')

fig.update_xaxes(title_text='Número de Delitos')
fig.update_yaxes(title_text='Municipios')

fig.update_layout(title='Número de Delitos por Municipio')
fig.update_layout(height=1000)

# Personalizando el texto en las barras
fig.update_traces(texttemplate='%{text}', textposition='outside')

fig.show()

In [962]:
# Agrupando los datos por municipio y suma el número de delitos
df_grouped = Incidencia_Delictiva_Mujeres_Municipio_df.groupby(['municipio'])['numero_delitos'].sum().reset_index()
df_grouped = df_grouped.sort_values(by='numero_delitos', ascending=False)

# Filtrando los 10 municipios con el número más alto de delitos
top_10_municipios = df_grouped.nlargest(10, 'numero_delitos')

# Filtrando los municipios que no están en el top 10
municipios_no_top10 = df_grouped[~df_grouped['municipio'].isin(top_10_municipios['municipio'])]

fig = px.bar(municipios_no_top10, x='numero_delitos', y='municipio', text='numero_delitos', orientation='h', color='municipio')

fig.update_xaxes(title_text='Número de Delitos')
fig.update_yaxes(title_text='Municipios', tickvals=lista_municipios, ticktext=[municipio for municipio in lista_municipios])

fig.update_layout(title='Número de Delitos por Municipio')
fig.update_layout(height=1000)

fig.update_traces(texttemplate='%{text}', textposition='outside')

fig.show()

In [963]:
mujer_temperatura_merge_total_df = mujer_temperatura_region_merge_df.groupby('temperatura_promedio')['numero_delitos'].sum().reset_index()

print(mujer_temperatura_merge_total_df)

      temperatura_promedio  numero_delitos
0                     6.07             0.0
1                     6.29             0.0
2                     6.47             1.0
3                     6.76             0.0
4                     6.79             0.0
...                    ...             ...
2327                 36.07            47.0
2328                 36.15            36.0
2329                 36.47             6.0
2330                 37.17            15.0
2331                 37.31            33.0

[2332 rows x 2 columns]


In [964]:
# Parámetro para el suavizado
ventana = 3

# Aplicando el suavizado por promedio móvil a 'numero_delitos'
mujer_temperatura_merge_total_df['numero_delitos_suavizados'] = mujer_temperatura_merge_total_df['numero_delitos'].rolling(window=ventana).mean()

# Definiendo una paleta de colores basada en los valores de temperatura
colores = np.array(mujer_temperatura_merge_total_df['temperatura_promedio'])

# Creando el gráfico
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=mujer_temperatura_merge_total_df['temperatura_promedio'],
    y=mujer_temperatura_merge_total_df['numero_delitos_suavizados'],
    mode='lines',
    marker=dict(color=colores, colorscale='Viridis', colorbar=dict(title='Temperatura')),
    name='Número Delitos'
))

fig.update_xaxes(title_text='Temperatura Promedio')
fig.update_yaxes(title_text='Número de Delitos')
fig.update_layout(title='Relación entre Temperatura y Número de Delitos')

fig.show()

In [965]:
mujer_temperatura_merge_df.groupby(['anio','mes','tipo_delito'])['temperatura_maxima'].max().reset_index()

,anio,mes,tipo_delito,temperatura_maxima
0,2015,Abril,Abandono,28.94
1,2015,Abril,Abuso Fetal,28.94
2,2015,Abril,Abuso Físico,28.94
3,2015,Abril,Abuso Psicológico,28.94
4,2015,Abril,Abuso Sexual,28.94
...,...,...,...,...
2535,2023,Mayo,Necesidad de Asistencia social,32.80
2536,2023,Mayo,Negligencia,32.80
2537,2023,Mayo,Omisión de Cuidados,32.80
2538,2023,Mayo,Violación,32.80


In [966]:
mujer_temperatura_merge_total_df = mujer_temperatura_merge_df.groupby(['temperatura_promedio','tipo_delito'])['numero_delitos'].sum().reset_index()

print(mujer_temperatura_merge_total_df)

      temperatura_promedio                     tipo_delito  numero_delitos
0                    12.86                        Abandono             0.0
1                    12.86                     Abuso Fetal             0.0
2                    12.86                    Abuso Físico            10.0
3                    12.86               Abuso Psicológico            35.0
4                    12.86                    Abuso Sexual             9.0
...                    ...                             ...             ...
2486                 32.33  Necesidad de Asistencia social             1.0
2487                 32.33                     Negligencia            48.0
2488                 32.33             Omisión de Cuidados             8.0
2489                 32.33                       Violación            13.0
2490                 32.33              Violencia Familiar           288.0

[2491 rows x 3 columns]


In [967]:
# Parámetro para el suavizado
ventana = 3

# Lista de años disponibles
tipos_de_delito = mujer_temperatura_merge_total_df['tipo_delito'].unique()

# Calculando el número de filas y columnas para el diseño de subgráficos
n_rows = (len(tipos_de_delito) + 1) // 2
n_cols = 2

# Creando un contenedor para los gráficos de líneas
fig = sp.make_subplots(rows=n_rows, cols=n_cols, subplot_titles=[f'Tipo de Delito: {tipo}' for tipo in tipos_de_delito])

# Ajustando el espacio entre subgráficos
fig.update_layout(height=300 * n_rows, showlegend=False)

color_scale = px.colors.sequential.Viridis

# Iteración a través de los años y para crear un gráfico de líneas para cada uno
for i, tipo in enumerate(tipos_de_delito):
    row, col = i // n_cols + 1, i % n_cols + 1  # +1 esto es para evitar la celda superior izquierda que contiene los títulos

     # Filtrando los datos para el tipo de delito actual
    datos_tipo_delito = mujer_temperatura_merge_total_df[mujer_temperatura_merge_total_df['tipo_delito'] == tipo]

    # Aplicando el suavizado por promedio móvil a 'numero_delitos'
    datos_tipo_delito['numero_delitos_suavizados'] = datos_tipo_delito['numero_delitos'].rolling(window=ventana).mean()

    #data = mujer_temperatura_merge_total_df[['region', tipo]].sort_values(by=tipo, ascending=False)
    title = f'Tipo de Delito: {tipo}'

    # Creando un gráfico de líneas
    line_fig = px.line(datos_tipo_delito, x="temperatura_promedio", y="numero_delitos_suavizados", title=title)
    
    fig.add_trace(line_fig.data[0], row=row, col=col)

    fig.update_xaxes(title_text="Temperatura Promedio", row=row, col=col)
    fig.update_yaxes(title_text="Número de Delitos", row=row, col=col)

fig.show()

C:\Users\lordm\AppData\Local\Temp\ipykernel_42244\2472050615.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\lordm\AppData\Local\Temp\ipykernel_42244\2472050615.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\lordm\AppData\Local\Temp\ipykernel_42244\2472050615.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [968]:
mujer_temperatura_merge_numeric_df = mujer_temperatura_merge_df.select_dtypes(include=['number'])
mujer_temperatura_merge_numeric_df

,anio,numero_delitos,temperatura_maxima,temperatura_minima,temperatura_promedio,precipitacion_total,lluvia_total,tiempo_precipitacion,minutos_luz,horas_luz
0,2015,167.0,21.95,8.42,14.76,3838.5,3836.7,4087.0,629.63,10.493833
1,2015,33.0,21.95,8.42,14.76,3838.5,3836.7,4087.0,629.63,10.493833
2,2015,14.0,21.95,8.42,14.76,3838.5,3836.7,4087.0,629.63,10.493833
3,2015,28.0,21.95,8.42,14.76,3838.5,3836.7,4087.0,629.63,10.493833
4,2015,9.0,21.95,8.42,14.76,3838.5,3836.7,4087.0,629.63,10.493833
...,...,...,...,...,...,...,...,...,...,...
2535,2022,1.0,21.73,8.15,14.48,935.1,928.0,2743.0,616.88,10.281333
2536,2022,0.0,21.73,8.15,14.48,935.1,928.0,2743.0,616.88,10.281333
2537,2022,0.0,21.73,8.15,14.48,935.1,928.0,2743.0,616.88,10.281333
2538,2022,0.0,21.73,8.15,14.48,935.1,928.0,2743.0,616.88,10.281333


In [969]:
# Creando un DataFrame de correlación
corr_data = mujer_temperatura_merge_numeric_df.corr()

# Creando el mapa de calor
fig = px.imshow(
    corr_data,
    color_continuous_scale='Viridis',  # Puedes ajustar la escala de colores aquí
    labels=dict(x="Variables", y="Variables", color="Correlación"),
)

# Agregando valores de correlación como texto
for i in range(len(corr_data.columns)):
    for j in range(len(corr_data.columns)):
        fig.add_annotation(
            text=str(round(corr_data.iloc[i, j], 2)),
            x=corr_data.columns[i],
            y=corr_data.columns[j],
            showarrow=False,
            font=dict(color="white"),
        )

# Configurando el diseño del gráfico
fig.update_layout(
    title='Mapa de Calor de Correlaciones',
    xaxis_side='bottom',
    height=800,
)

fig.show()

In [970]:
numero_tipo_delitos

,tipo_delito,total_delitos
0,Abandono,189.0
1,Abuso Fetal,104.0
2,Abuso Físico,1442.0
3,Abuso Psicológico,2055.0
4,Abuso Sexual,636.0
5,Abusos Deshonestos,838.0
6,Acoso escolar(Bullying),43.0
7,Acoso laboral,83.0
8,Alcoholismo,3.0
9,Corrupción de Menores,63.0


In [971]:
delitos_order = numero_tipo_delitos.sort_values(by='total_delitos', ascending=False)
top_15_delitos_serie = delitos_order[:15]['tipo_delito'].unique()
print(top_15_delitos_serie)

['Violencia Familiar' 'Incumpl. obligaciones familiares' 'Negligencia'
 'Omisión de Cuidados' 'Abuso Psicológico' 'Abuso Físico' 'Violación'
 'Abusos Deshonestos' 'Abuso Sexual' 'Necesidad de Asistencia social'
 'Homicidio Doloso' 'Mujeres Desaparecidas/Extraviadas' 'Discriminación'
 'Feminicidio' 'Hostigamiento Sexual']


In [972]:
serie_tiempo_delitos(Incidencia_Delictiva_Mujeres_Tipo_Delito_df,top_15_delitos_serie)

In [981]:
df_top = Incidencia_Delictiva_Mujeres_Municipio_df.groupby(['region', 'municipio'])['numero_delitos'].sum().reset_index().sort_values(by='numero_delitos', ascending=False)

df_top_cn = df_top[df_top['region']=='Noraeste']

print(df_top_cn[:3])

      region              municipio  numero_delitos
17  Noraeste  San Luis Río Colorado          3460.0
12  Noraeste                Caborca          1814.0
16  Noraeste         Puerto Peñasco           887.0
